# PySparkでデータ前処理100本ノックをやってみる

* 参考
    * [データサイエンス100本ノック（構造化データ加工編）](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess/tree/master/docker/work)
    * [PySparkでCSVファイルを読み込む方法](https://docs.kanaries.net/ja/articles/read-csv-dataframe)
    * [PySparkでparquetファイルを読み込む方法](https://data-analysis-stats.jp/spark/pyspark%E3%81%A7%E3%83%87%E3%83%BC%E3%82%BF%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BF/)
    * [PolarsでCSVファイルをParquetに変換する方法](https://medium.com/@umesh.nagar92x/efficient-conversion-of-massive-csv-files-to-parquet-format-using-pandas-dask-duck-db-and-polars-5c998c47b43d)

In [1]:
from glob import glob

import polars as pl
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.functions import vector_to_array
import numpy as np

# Create a SparkSession。pythonからsparkを使う場合、セッションの作成が必要。
spark = SparkSession.builder.appName("Testing PySpark Example").getOrCreate()

# デフォルトのログレベルだと大量にログが出力されるので限定する。
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/21 14:01:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Polarsを用いて入力ファイル(csv)をParquetに変換する
PySparkはcsvを読むのが苦手なので、Parquetファイルに変換する

In [28]:
csv_files = glob('../../../100knocks-preprocess/docker/work/data/*.csv')
csv_files

['../../../100knocks-preprocess/docker/work/data/store.csv',
 '../../../100knocks-preprocess/docker/work/data/receipt.csv',
 '../../../100knocks-preprocess/docker/work/data/category.csv',
 '../../../100knocks-preprocess/docker/work/data/product.csv',
 '../../../100knocks-preprocess/docker/work/data/customer.csv',
 '../../../100knocks-preprocess/docker/work/data/geocode.csv']

In [29]:
dtypes = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str,
    'application_date': str,
    'birth_day': pl.Date
}

# それぞれcsvを読み込んでからparquetとして出力
for file in csv_files:
    df = pl.read_csv(file, dtypes=dtypes)

    # fileの絶対パスの拡張子をparquetにして出力
    output_path = file.split(".csv")[0] + ".parquet"
    df.write_parquet(output_path)


## PySparkでParquetファイルを読み込み

In [79]:
# 各データ読み込み
df_receipt = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/receipt.parquet")

# 店舗データ
df_store = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/store.parquet")

# 顧客データ
df_customer = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/customer.parquet")

# 製品データ
df_product = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/product.parquet")

# 製品データ
df_category = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/category.parquet")

# ジオコードデータ
df_geocode = spark.read.parquet("../../../100knocks-preprocess/docker/work/data/geocode.parquet")

In [4]:
df_receipt.printSchema()

root
 |-- sales_ymd: long (nullable = true)
 |-- sales_epoch: long (nullable = true)
 |-- store_cd: string (nullable = true)
 |-- receipt_no: long (nullable = true)
 |-- receipt_sub_no: long (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_cd: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- amount: long (nullable = true)



In [66]:
df_store.printSchema()

root
 |-- store_cd: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- prefecture_cd: string (nullable = true)
 |-- prefecture: string (nullable = true)
 |-- address: string (nullable = true)
 |-- address_kana: string (nullable = true)
 |-- tel_no: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- floor_area: double (nullable = true)



# 100本ノック

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [44]:
df_receipt.show(10)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
| 20190205| 1549324800|  S14042|      1132|             1|ZZ000000000000|P050301001|       1|    25|
| 20180821| 1534809600|  S14025|      1102|             2|CS025415000050|P060102007|       1|    90|
| 20190605| 1559692800|  S13003|      1112|             1|CS003515000195|P050102002|       1|   138|
| 20181205| 1543968000|  S14024|      1102|             2|CS024514000042|P080101005|       

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [45]:
df_receipt.select(
    "sales_ymd",
    "customer_id",
    "product_cd",
    "amount"
).show(10)

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20181103|CS006214000001|P070305012|   158|
| 20181118|CS008415000097|P070701017|    81|
| 20170712|CS028414000014|P060101005|   170|
| 20190205|ZZ000000000000|P050301001|    25|
| 20180821|CS025415000050|P060102007|    90|
| 20190605|CS003515000195|P050102002|   138|
| 20181205|CS024514000042|P080101005|    30|
| 20190922|CS040415000178|P070501004|   128|
| 20170504|ZZ000000000000|P071302010|   770|
| 20191010|CS027514000015|P071101003|   680|
+---------+--------------+----------+------+
only showing top 10 rows



---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、  
> 10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [48]:
df_receipt.select(
    F.col("sales_ymd").alias("sales_date"),
    "customer_id",
    "product_cd",
    "amount"
).show(10)

+----------+--------------+----------+------+
|sales_date|   customer_id|product_cd|amount|
+----------+--------------+----------+------+
|  20181103|CS006214000001|P070305012|   158|
|  20181118|CS008415000097|P070701017|    81|
|  20170712|CS028414000014|P060101005|   170|
|  20190205|ZZ000000000000|P050301001|    25|
|  20180821|CS025415000050|P060102007|    90|
|  20190605|CS003515000195|P050102002|   138|
|  20181205|CS024514000042|P080101005|    30|
|  20190922|CS040415000178|P070501004|   128|
|  20170504|ZZ000000000000|P071302010|   770|
|  20191010|CS027514000015|P071101003|   680|
+----------+--------------+----------+------+
only showing top 10 rows



---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [51]:
df_receipt.select(
    "sales_ymd",
    "customer_id",
    "product_cd",
    "amount"
).filter(
    F.col("customer_id") == "CS018205000001"
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20180414|CS018205000001|P060104007|   600|
| 20170614|CS018205000001|P050206001|   990|
| 20170614|CS018205000001|P060702015|   108|
| 20190216|CS018205000001|P071005024|   102|
| 20180414|CS018205000001|P071101002|   278|
| 20190226|CS018205000001|P070902035|   168|
| 20190924|CS018205000001|P060805001|   495|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
| 20190216|CS018205000001|P040101002|   218|
| 20190924|CS018205000001|P091503001|   280|
+---------+--------------+----------+------+



---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、  
> 売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [53]:
df_receipt.select(
    "sales_ymd",
    "customer_id",
    "product_cd",
    "amount"
).filter(
    (F.col("customer_id") == "CS018205000001") & (F.col("amount") >= 1000 )
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+



---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、  
> 以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [59]:
cols = ["sales_ymd","customer_id","product_cd","quantity","amount"]
df_receipt.select(
    cols
).filter(
    (F.col("customer_id") == "CS018205000001") &
    ((F.col("amount") >= 1000) | (F.col("quantity") >= 5))
).show()

+---------+--------------+----------+--------+------+
|sales_ymd|   customer_id|product_cd|quantity|amount|
+---------+--------------+----------+--------+------+
| 20180911|CS018205000001|P071401012|       1|  2200|
| 20180414|CS018205000001|P060104007|       6|   600|
| 20170614|CS018205000001|P050206001|       5|   990|
| 20190226|CS018205000001|P071401020|       1|  2200|
| 20180911|CS018205000001|P071401005|       1|  1100|
+---------+--------------+----------+--------+------+



---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、  
> 以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [60]:
cols = ["sales_ymd","customer_id","product_cd","amount"]
df_receipt.select(
    cols
).filter(
    (F.col("customer_id") == "CS018205000001") &
    (F.col("amount").between(1000, 2000))
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+



---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、  
> 以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [61]:
cols = ["sales_ymd","customer_id","product_cd","amount"]
df_receipt.select(
    cols
).filter(
    (F.col("customer_id") == "CS018205000001") &
    (F.col("product_cd") != "P071401019")
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20180414|CS018205000001|P060104007|   600|
| 20170614|CS018205000001|P050206001|   990|
| 20170614|CS018205000001|P060702015|   108|
| 20190216|CS018205000001|P071005024|   102|
| 20180414|CS018205000001|P071101002|   278|
| 20190226|CS018205000001|P070902035|   168|
| 20190924|CS018205000001|P060805001|   495|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
| 20190216|CS018205000001|P040101002|   218|
| 20190924|CS018205000001|P091503001|   280|
+---------+--------------+----------+------+



---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [70]:
# 問題文の結果
df_store.filter(
    ~((F.col("prefecture_cd") == "13") | (F.col("floor_area") > 900))
).show()

+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                           address|                          address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|  S14046|  北山田店|           14|  神奈川県|  神奈川県横浜市都筑区北山田一丁目| カナガワケンヨコハマシツヅキクキタ...|045-123-4049| 139.5916|35.56189|     831.0|
|  S14011|日吉本町店|           14|  神奈川県|神奈川県横浜市港北区日吉本町四丁目| カナガワケンヨコハマシコウホククヒ...|045-123-4033| 139.6316|35.54655|     890.0|
|  S12013|  習志野店|           12|    千葉県|          千葉県習志野市芝園一丁目|チバケンナラシノシシバゾノイッチョウメ|047-123-4002|  140.022|35.66122|     808.0|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+-----

In [71]:
# 書き換え
df_store.filter(
    ((F.col("prefecture_cd") != "13") & (F.col("floor_area") <= 900))
).show()

+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                           address|                          address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|  S14046|  北山田店|           14|  神奈川県|  神奈川県横浜市都筑区北山田一丁目| カナガワケンヨコハマシツヅキクキタ...|045-123-4049| 139.5916|35.56189|     831.0|
|  S14011|日吉本町店|           14|  神奈川県|神奈川県横浜市港北区日吉本町四丁目| カナガワケンヨコハマシコウホククヒ...|045-123-4033| 139.6316|35.54655|     890.0|
|  S12013|  習志野店|           12|    千葉県|          千葉県習志野市芝園一丁目|チバケンナラシノシシバゾノイッチョウメ|047-123-4002|  140.022|35.66122|     808.0|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+-----

---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [74]:
df_store.filter(
    F.col("store_cd").startswith("S14")
).show(10)

+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|  store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S14010|      菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|    阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4043| 139.4961|35.45918|    1495.0|
|  S14036|相模原中央店|           14|  神奈川県|            神奈川県相模原市中央二丁目|   カナガワケンサガミハラシチュウオウ...|042-123-4045| 139.3716|35.57327|    1679.0|
|  S14040|    長津田店|           14|  神奈川県|神奈川県横浜市緑区長津田みなみ台五丁目|   カナガワケンヨコハマシミドリクナガ...|045-1

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [77]:
df_customer.filter(
    F.col("customer_id").endswith("1")
).show(10)

+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                          address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|CS037613000071|    六角 雅彦|        9|  不明|1952-04-01| 66| 136-0076|       東京都江東区南砂**********|              S13037|        20150414|0-00000000-0|
|CS028811000001|  堀井 かおり|        1|  女性|1933-03-27| 86| 245-0016| 神奈川県横浜市泉区和泉町*****...|              S14028|        20160115|0-00000000-0|
|CS040412000191|    川井 郁恵|        1|  女性|1977-01-05| 42| 226-0021|神奈川県横浜市緑区北八朔町****...|              S14040|        20151101|1-20091025-4|
|CS028314000011|  小菅 あおい|        1|  女性|1983-11-26| 35| 246-0038| 神奈川県横浜市瀬谷区宮沢*****...|              S14

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [79]:
df_store.filter(
    F.col("address").contains("横浜市")
).show()

+--------+----------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S14010|    菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|  阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4043| 139.4961|35.45918|    1495.0|
|  S14040|  長津田店|           14|  神奈川県|神奈川県横浜市緑区長津田みなみ台五丁目|   カナガワケンヨコハマシミドリクナガ...|045-123-4046| 139.4994|35.52398|    1548.0|
|  S14050|阿久和西店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4053| 139.4961

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [86]:
# 1. F.regexpを使う方法。F.litで正規表現の列表現を指定する必要がある。
df_customer.filter(
    F.regexp("status_cd", F.lit("^[A-F]"))
).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS031415000172|宇多田 貴美子|        1|  女性|1976-10-04| 42| 151-0053|      東京都渋谷区代々木**********|              S13031|        20150529|D-20100325-C|
|CS015414000103|    奥野 陽子|        1|  女性|1977-08-09| 41| 136-0073|        東京都江東区北砂**********|              S13015|        20150722|B-20100609-B|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS029415000023|    梅田 里穂|        1|  女性|1976-01-17| 43| 279-0043|      千葉県浦安市富士見**********|     

In [88]:
# 2. SQLっぽいマッチのさせ方。こっちの方が簡単に書ける。
df_customer.filter(
    F.col("status_cd").rlike("^[A-F]")
).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS031415000172|宇多田 貴美子|        1|  女性|1976-10-04| 42| 151-0053|      東京都渋谷区代々木**********|              S13031|        20150529|D-20100325-C|
|CS015414000103|    奥野 陽子|        1|  女性|1977-08-09| 41| 136-0073|        東京都江東区北砂**********|              S13015|        20150722|B-20100609-B|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS029415000023|    梅田 里穂|        1|  女性|1976-01-17| 43| 279-0043|      千葉県浦安市富士見**********|     

In [93]:
# containsではうまくまっちしない
df_customer.filter(
    F.col("status_cd").contains(r"^[A-F]")
).show(10)

+-----------+-------------+---------+------+---------+---+---------+-------+--------------------+----------------+---------+
|customer_id|customer_name|gender_cd|gender|birth_day|age|postal_cd|address|application_store_cd|application_date|status_cd|
+-----------+-------------+---------+------+---------+---+---------+-------+--------------------+----------------+---------+
+-----------+-------------+---------+------+---------+---+---------+-------+--------------------+----------------+---------+



---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [91]:
df_customer.filter(
    F.col("status_cd").rlike("[1-9]$")
).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS001215000145|    田崎 美紀|        1|  女性|1995-03-29| 24| 144-0055|      東京都大田区仲六郷**********|              S13001|        20170605|6-20090929-2|
|CS033513000180|      安斎 遥|        1|  女性|1962-07-11| 56| 241-0823|  神奈川県横浜市旭区善部町*****...|              S14033|        20150728|6-20080506-5|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS040412000191|    川井 郁恵|        1|  女性|1977-01-05| 42| 226-0021| 神奈川県横浜市緑区北八朔町****...|          

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、  
> 末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [96]:
df_customer.filter(
    (F.col("status_cd").rlike("^[A-F].*[1-9]$")) 
).show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS022513000105|  島村 貴美子|        1|  女性|1962-03-12| 57| 249-0002|    神奈川県逗子市山の根**********|              S14022|        20150320|A-20091115-7|
|CS001515000096|    水野 陽子|        9|  不明|1960-11-29| 58| 144-0053|    東京都大田区蒲田本町**********|              S13001|        20150614|A-20100724-7|
|CS013615000053|    西脇 季衣|        1|  女性|1953-10-18| 65| 261-0026|  千葉県千葉市美浜区幕張西*****...|          

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [110]:
df_store.filter(
    F.col("tel_no").rlike("^[0-9]{3}-[0-9]{3}-[0-9]{4}$")
).show()

+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|  store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S12014|    千草台店|           12|    千葉県|        千葉県千葉市稲毛区千草台一丁目|   チバケンチバシイナゲクチグサダイイ...|043-123-4003|  140.118|35.63559|    1698.0|
|  S13002|    国分寺店|           13|    東京都|              東京都国分寺市本多二丁目|トウキョウトコクブンジシホンダニチョウメ|042-123-4008| 139.4802|35.70566|    1735.0|
|  S14010|      菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|    阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [116]:
df_customer.sort("birth_day").show(10)

+--------------+-------------+---------+------+----------+---+---------+--------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                         address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+--------------------------------+--------------------+----------------+------------+
|CS003813000014|  村山 菜々美|        1|  女性|1928-11-26| 90| 182-0007|    東京都調布市菊野台**********|              S13003|        20160214|0-00000000-0|
|CS026813000004|    吉村 朝陽|        1|  女性|1928-12-14| 90| 251-0043| 神奈川県藤沢市辻堂元町******...|              S14026|        20150723|0-00000000-0|
|CS018811000003|    熊沢 美里|        1|  女性|1929-01-07| 90| 204-0004|      東京都清瀬市野塩**********|              S13018|        20150403|0-00000000-0|
|CS027803000004|    内村 拓郎|        0|  男性|1929-01-12| 90| 251-0031|神奈川県藤沢市鵠沼藤が谷*****...|              S140

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [117]:
# 降順にソート
df_customer.sort("birth_day", ascending=False).show(10)

+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                          address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|CS035114000004|    大村 美里|        1|  女性|2007-11-25| 11| 156-0053|       東京都世田谷区桜**********|              S13035|        20150619|6-20091205-6|
|CS022103000002|  福山 はじめ|        9|  不明|2007-10-02| 11| 249-0006|     神奈川県逗子市逗子**********|              S14022|        20160909|0-00000000-0|
|CS002113000009|  柴田 真悠子|        1|  女性|2007-09-17| 11| 184-0014|  東京都小金井市貫井南町******...|              S13002|        20160304|0-00000000-0|
|CS004115000014|    松井 京子|        1|  女性|2007-08-09| 11| 165-0031|     東京都中野区上鷺宮**********|         

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。  
> 項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [131]:
w = Window.orderBy(F.desc("amount"))

df_receipt.select(
    "customer_id",
    "amount",
    F.dense_rank().over(w).alias("順位")
).show(10)

+--------------+------+----+
|   customer_id|amount|順位|
+--------------+------+----+
|CS011415000006| 10925|   1|
|ZZ000000000000|  6800|   2|
|CS028605000002|  5780|   3|
|CS015515000034|  5480|   4|
|ZZ000000000000|  5480|   4|
|ZZ000000000000|  5480|   4|
|ZZ000000000000|  5440|   5|
|CS021515000089|  5440|   5|
|ZZ000000000000|  5280|   6|
|CS017414000114|  5280|   6|
+--------------+------+----+
only showing top 10 rows



dense_rankによって同着がいた場合、次の順位は同着＋１の順位になる。そうしたくない場合はrankを用いること。  
使い方はSQLのような感じで.over(window)のように使用する。windowには対象カラムに対するソート操作を定義する。(polarsと異なり、結構めんどくさい。)  
また、pyspark.sqlのWindowクラスをimportしておく必要があることに注意。    
* [dense_rankについて](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.dense_rank.html?highlight=dense_rank)
* [Window関数について](https://www.niandc.co.jp/tech/20220613_2187/)
OVERはこれからWindow関数を使うサインだと考えればよいみたい。

---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。  
> 項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [134]:
w = Window.orderBy(F.desc("amount"))

df_receipt.select(
    "customer_id",
    "amount",
    F.row_number().over(w).alias("順位")
).show()

+--------------+------+----+
|   customer_id|amount|順位|
+--------------+------+----+
|CS011415000006| 10925|   1|
|ZZ000000000000|  6800|   2|
|CS028605000002|  5780|   3|
|CS015515000034|  5480|   4|
|ZZ000000000000|  5480|   5|
|ZZ000000000000|  5480|   6|
|ZZ000000000000|  5440|   7|
|CS021515000089|  5440|   8|
|ZZ000000000000|  5280|   9|
|CS028515000029|  5280|  10|
|ZZ000000000000|  5280|  11|
|CS035615000040|  5280|  12|
|ZZ000000000000|  5280|  13|
|CS030515000189|  5280|  14|
|CS017414000114|  5280|  15|
|ZZ000000000000|  5280|  16|
|CS035414000024|  5280|  17|
|CS039415000073|  5280|  18|
|CS021515000089|  5280|  19|
|CS038415000071|  5280|  20|
+--------------+------+----+
only showing top 20 rows



同着でも構わず連続の順位をつけたい場合はrow_numberを用いる。ここはSQLと一緒。

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [135]:
df_receipt.count()

104681

In [136]:
len(df_receipt)

TypeError: object of type 'DataFrame' has no len()

pysparkのDataframeにはlenは使えないので注意。

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [163]:
df_receipt.select(
    F.count_distinct("customer_id")
).show()

+---------------------------+
|count(DISTINCT customer_id)|
+---------------------------+
|                       8307|
+---------------------------+



polarsのuniqueではなくcount_distinctでユニークな件数をカウントすることに注意

In [250]:
# 単一の数字としてほしい場合は一度collectしてから下記のように取り出す。
df_receipt.select(
    F.count_distinct("customer_id")
).collect()[0]["count(DISTINCT customer_id)"]

8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [7]:
df_receipt.groupBy("store_cd").agg(
    F.sum(F.col("amount")),
    F.sum(F.col("quantity"))
).sort("store_cd").show()

+--------+-----------+-------------+
|store_cd|sum(amount)|sum(quantity)|
+--------+-----------+-------------+
|  S12007|     638761|         2099|
|  S12013|     787513|         2425|
|  S12014|     725167|         2358|
|  S12029|     794741|         2555|
|  S12030|     684402|         2403|
|  S13001|     811936|         2347|
|  S13002|     727821|         2340|
|  S13003|     764294|         2197|
|  S13004|     779373|         2390|
|  S13005|     629876|         2004|
|  S13008|     809288|         2491|
|  S13009|     808870|         2486|
|  S13015|     780873|         2248|
|  S13016|     793773|         2432|
|  S13017|     748221|         2376|
|  S13018|     790535|         2562|
|  S13019|     827833|         2541|
|  S13020|     796383|         2383|
|  S13031|     705968|         2336|
|  S13032|     790501|         2491|
+--------+-----------+-------------+
only showing top 20 rows



---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [15]:
df_receipt.groupBy("customer_id").agg(F.max("sales_ymd")).show(10)

+--------------+--------------+
|   customer_id|max(sales_ymd)|
+--------------+--------------+
|CS024415000195|      20190907|
|CS011515000044|      20181226|
|CS017415000245|      20190615|
|CS011512000113|      20190829|
|CS025414000093|      20191016|
|CS010605000007|      20181009|
|CS011415000097|      20180909|
|CS018515000090|      20190717|
|CS015415000222|      20190928|
|CS038605000003|      20190910|
+--------------+--------------+
only showing top 10 rows



---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [16]:
df_receipt.groupBy("customer_id").agg(F.min("sales_ymd")).show(10)

+--------------+--------------+
|   customer_id|min(sales_ymd)|
+--------------+--------------+
|CS024415000195|      20170119|
|CS011515000044|      20170505|
|CS017415000245|      20180110|
|CS011512000113|      20181213|
|CS025414000093|      20170806|
|CS010605000007|      20170125|
|CS011415000097|      20170108|
|CS018515000090|      20170131|
|CS015415000222|      20170313|
|CS038605000003|      20170321|
+--------------+--------------+
only showing top 10 rows



---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、  
> 両者が異なるデータを10件表示せよ。

In [21]:
df_receipt.groupBy("customer_id").agg(
    F.max("sales_ymd").alias("newest"),
    F.min("sales_ymd").alias("oldest")
).filter(
    F.col("newest") != F.col("oldest")
).show(10)

+--------------+--------+--------+
|   customer_id|  newest|  oldest|
+--------------+--------+--------+
|CS024415000195|20190907|20170119|
|CS011515000044|20181226|20170505|
|CS017415000245|20190615|20180110|
|CS011512000113|20190829|20181213|
|CS025414000093|20191016|20170806|
|CS010605000007|20181009|20170125|
|CS011415000097|20180909|20170108|
|CS018515000090|20190717|20170131|
|CS015415000222|20190928|20170313|
|CS038605000003|20190910|20170321|
+--------------+--------+--------+
only showing top 10 rows



---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [23]:
df_receipt.groupBy("store_cd").agg(
    F.mean("amount").alias("amount_mean")
).sort("amount_mean", ascending=False).show(5)

+--------+------------------+
|store_cd|       amount_mean|
+--------+------------------+
|  S13052|402.86746987951807|
|  S13015|351.11196043165467|
|  S13003| 350.9155188246097|
|  S14010|348.79126213592235|
|  S13001| 348.4703862660944|
+--------+------------------+
only showing top 5 rows



---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、  
> 降順でTOP5を表示せよ。

In [4]:
df_receipt.groupBy("store_cd").agg(
    F.median("amount").alias("amount_median")
).sort("amount_median", ascending=False).show(5)

+--------+-------------+
|store_cd|amount_median|
+--------+-------------+
|  S13052|        190.0|
|  S14010|        188.0|
|  S14050|        185.0|
|  S14040|        180.0|
|  S13003|        180.0|
+--------+-------------+
only showing top 5 rows



---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [8]:
df_receipt.groupBy("store_cd").agg(
    F.mode("product_cd").alias("product_cdの最頻値")
).sort("store_cd").show(10)

+--------+------------------+
|store_cd|product_cdの最頻値|
+--------+------------------+
|  S12007|        P060303001|
|  S12013|        P060303001|
|  S12014|        P060303001|
|  S12029|        P060303001|
|  S12030|        P060303001|
|  S13001|        P060303001|
|  S13002|        P060303001|
|  S13003|        P071401001|
|  S13004|        P060303001|
|  S13005|        P040503001|
+--------+------------------+
only showing top 10 rows



---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [13]:
df_receipt.groupBy("store_cd").agg(
    F.var_pop(F.col("amount")).alias("amount_var")
).sort("amount_var", ascending=False).show(5)

+--------+------------------+
|store_cd|        amount_var|
+--------+------------------+
|  S13052|440088.70131126925|
|  S14011| 306314.5581638889|
|  S14034| 296920.0810112844|
|  S13001|295431.99332903494|
|  S13015| 295294.3611159408|
+--------+------------------+
only showing top 5 rows



---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [14]:
df_receipt.groupBy("store_cd").agg(
    F.std(F.col("amount")).alias("amount_std")
).sort("amount_std", ascending=False).show(5)

+--------+-----------------+
|store_cd|       amount_std|
+--------+-----------------+
|  S13052|664.7279537722436|
|  S14011|553.5722558361905|
|  S14034| 545.039808399406|
|  S13001|543.6532374756101|
|  S13015|543.5321490821435|
+--------+-----------------+
only showing top 5 rows



---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [26]:
df_receipt.select(
    F.percentile("amount", 0.).alias("amount_0%tile"),
    F.percentile("amount", 0.25).alias("amount_25%tile"),
    F.percentile("amount", 0.50).alias("amount_50%tile"),
    F.percentile("amount", 0.75).alias("amount_75%tile"),
    F.percentile("amount", 1.0).alias("amount_100%tile"),
).show()

+-------------+--------------+--------------+--------------+---------------+
|amount_0%tile|amount_25%tile|amount_50%tile|amount_75%tile|amount_100%tile|
+-------------+--------------+--------------+--------------+---------------+
|         10.0|         102.0|         170.0|         288.0|        10925.0|
+-------------+--------------+--------------+--------------+---------------+



## 別解1：　リスト内包表記でexprを定義する
同じようなexprを何度も書くのが面倒な時に有効。

In [39]:
expr = [F.percentile("amount", ptile).alias(f"amount_{ int(ptile*100) }%tile") for ptile in np.arange(0, 1.25, 0.25)]
expr

[Column<'percentile(amount, 0.0, 1) AS `amount_0%tile`'>,
 Column<'percentile(amount, 0.25, 1) AS `amount_25%tile`'>,
 Column<'percentile(amount, 0.5, 1) AS `amount_50%tile`'>,
 Column<'percentile(amount, 0.75, 1) AS `amount_75%tile`'>,
 Column<'percentile(amount, 1.0, 1) AS `amount_100%tile`'>]

In [40]:
df_receipt.select(
    expr
).show()

+-------------+--------------+--------------+--------------+---------------+
|amount_0%tile|amount_25%tile|amount_50%tile|amount_75%tile|amount_100%tile|
+-------------+--------------+--------------+--------------+---------------+
|         10.0|         102.0|         170.0|         288.0|        10925.0|
+-------------+--------------+--------------+--------------+---------------+



## 別解2：　F.percentileにリストを渡す

In [48]:
df_receipt.select(
    F.percentile("amount", np.arange(0, 1.25, 0.25)),
).collect()[0][0]

[10.0, 102.0, 170.0, 288.0, 10925.0]

上記のように書けばそれぞれの分位数を取得することが出来る。

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [52]:
df_receipt.groupBy("store_cd").agg(
    F.mean("amount").alias("amount_mean")
).filter(
    F.col("amount_mean") >= 330
).sort("store_cd").show()

+--------+------------------+
|store_cd|       amount_mean|
+--------+------------------+
|  S12013|330.19412997903567|
|  S13001| 348.4703862660944|
|  S13003| 350.9155188246097|
|  S13004|330.94394904458596|
|  S13015|351.11196043165467|
|  S13019|330.20861587554845|
|  S13020|  337.879932117098|
|  S13052|402.86746987951807|
|  S14010|348.79126213592235|
|  S14011| 335.7183333333333|
|  S14026|332.34058847239015|
|  S14045| 330.0820734341253|
|  S14047| 330.0770731707317|
+--------+------------------+



---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。  
> ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [70]:
df_receipt.filter(
    ~F.col("customer_id").startswith("Z")   
).groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).select(
    F.mean("total_amount")
).collect()[0][0]

2547.742234529256

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、  
> 平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [115]:
amount_mean_all = \
(
    df_receipt.filter(
    ~F.col("customer_id").startswith("Z")   
    ).groupBy("customer_id").agg(
        F.sum("amount").alias("total_amount")
    ).select(
        F.mean("total_amount")
    ).collect()[0][0]
)
amount_mean_all

2547.742234529256

In [114]:
df_receipt.filter(
    (~F.col("customer_id").startswith("Z")) &
    (F.col("amount") >= amount_mean_all)
).sort("amount").show(10)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20190903| 1567468800|  S13015|      1112|             2|CS015415000080|P090901062|       1|  2560|
| 20190615| 1560556800|  S13039|      1102|             2|CS039415000129|P090404061|       1|  2580|
| 20171125| 1511568000|  S14006|      1112|             2|CS006513000175|P090404065|       1|  2580|
| 20170306| 1488758400|  S14006|      1152|             2|CS006414000029|P090901063|       1|  2580|
| 20181030| 1540857600|  S13020|      1152|             2|CS020415000008|P090404065|       1|  2580|
| 20190515| 1557878400|  S13020|      1142|             2|CS020415000250|P071201013|       1|  2580|
| 20180906| 1536192000|  S13035|       112|             1|CS035212000007|P071201013|       

Polarsではpl.litで簡単に単一の平均値のカラムを作成出来るが、pysparkでは一度数字として変数に格納する必要がある模様。  
下記の書き方ではうまくいかない。これは、mean関数が集約関数であり、列を指定して計算する必要があるため。

In [132]:
(
    df_receipt.filter(
    ~F.col("customer_id").startswith("Z")   
    ).groupBy("customer_id").agg(
        F.sum("amount").alias("total_amount")
    ).withColumn(
        "mean_amount_all",
        F.lit(F.mean("total_amount"))
    ).show()
)

AnalysisException: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses.;
Aggregate [customer_id#5, total_amount#3270L, avg(total_amount#3270L) AS mean_amount_all#3274]
+- Aggregate [customer_id#5], [customer_id#5, sum(amount#8L) AS total_amount#3270L]
   +- Filter NOT StartsWith(customer_id#5, Z)
      +- Relation [sales_ymd#0L,sales_epoch#1L,store_cd#2,receipt_no#3L,receipt_sub_no#4L,customer_id#5,product_cd#6,quantity#7L,amount#8L] parquet


---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、  
> レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [136]:
df_receipt.show(3)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
only showing top 3 rows



In [135]:
df_store.show(3)

+--------+----------+-------------+----------+------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                       address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S12014|  千草台店|           12|    千葉県|千葉県千葉市稲毛区千草台一丁目|   チバケンチバシイナゲクチグサダイイ...|043-123-4003|  140.118|35.63559|    1698.0|
|  S13002|  国分寺店|           13|    東京都|      東京都国分寺市本多二丁目|トウキョウトコクブンジシホンダニチョウメ|042-123-4008| 139.4802|35.70566|    1735.0|
|  S14010|    菊名店|           14|  神奈川県|神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
+--------+----------+-------------+----------+------------------------------+----------------------------------------+------------+---------+----

In [142]:
df_receipt.join(
    df_store.select("store_cd", "store_name"),
    "store_cd",
    "inner"
).show(10)

+--------+---------+-----------+----------+--------------+--------------+----------+--------+------+----------+
|store_cd|sales_ymd|sales_epoch|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|store_name|
+--------+---------+-----------+----------+--------------+--------------+----------+--------+------+----------+
|  S14006| 20181103| 1541203200|       112|             1|CS006214000001|P070305012|       1|   158|  葛が谷店|
|  S13008| 20181118| 1542499200|      1132|             2|CS008415000097|P070701017|       1|    81|    成城店|
|  S14028| 20170712| 1499817600|      1102|             1|CS028414000014|P060101005|       1|   170|  二ツ橋店|
|  S14042| 20190205| 1549324800|      1132|             1|ZZ000000000000|P050301001|       1|    25|  新山下店|
|  S14025| 20180821| 1534809600|      1102|             2|CS025415000050|P060102007|       1|    90|    大和店|
|  S13003| 20190605| 1559692800|      1112|             1|CS003515000195|P050102002|       1|   138|    狛江店|
|  S14024| 20

---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、  
> 商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [139]:
df_product.show(3)

+----------+-----------------+------------------+-----------------+----------+---------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|
+----------+-----------------+------------------+-----------------+----------+---------+
|P040101001|               04|              0401|           040101|       198|      149|
|P040101002|               04|              0401|           040101|       218|      164|
|P040101003|               04|              0401|           040101|       230|      173|
+----------+-----------------+------------------+-----------------+----------+---------+
only showing top 3 rows



In [141]:
df_category.show(3)

+-----------------+-------------------+------------------+--------------------+-----------------+-------------------+
|category_major_cd|category_major_name|category_medium_cd|category_medium_name|category_small_cd|category_small_name|
+-----------------+-------------------+------------------+--------------------+-----------------+-------------------+
|               04|               惣菜|              0401|              御飯類|           040101|             弁当類|
|               04|               惣菜|              0401|              御飯類|           040102|             寿司類|
|               04|               惣菜|              0402|              佃煮類|           040201|         魚介佃煮類|
+-----------------+-------------------+------------------+--------------------+-----------------+-------------------+
only showing top 3 rows



In [3]:
df_product.join(
    df_category.select("category_small_cd", "category_small_name"),
    "category_small_cd",
    "inner"
).show(10)

+-----------------+----------+-----------------+------------------+----------+---------+-------------------+
|category_small_cd|product_cd|category_major_cd|category_medium_cd|unit_price|unit_cost|category_small_name|
+-----------------+----------+-----------------+------------------+----------+---------+-------------------+
|           040101|P040101001|               04|              0401|       198|      149|             弁当類|
|           040101|P040101002|               04|              0401|       218|      164|             弁当類|
|           040101|P040101003|               04|              0401|       230|      173|             弁当類|
|           040101|P040101004|               04|              0401|       248|      186|             弁当類|
|           040101|P040101005|               04|              0401|       268|      201|             弁当類|
|           040101|P040101006|               04|              0401|       298|      224|             弁当類|
|           040101|P040101007|       

---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。  
> ただし、売上実績がない顧客については売上金額を0として表示させること。  
> また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [4]:
df_customer.show(3)

+--------------+-------------+---------+------+----------+---+---------+------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                       address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+------------------------------+--------------------+----------------+------------+
|CS021313000114|  大野 あや子|        1|  女性|1981-04-29| 37| 259-1113|神奈川県伊勢原市粟窪**********|              S14021|        20150905|0-00000000-0|
|CS037613000071|    六角 雅彦|        9|  不明|1952-04-01| 66| 136-0076|    東京都江東区南砂**********|              S13037|        20150414|0-00000000-0|
|CS031415000172|宇多田 貴美子|        1|  女性|1976-10-04| 42| 151-0053|  東京都渋谷区代々木**********|              S13031|        20150529|D-20100325-C|
+--------------+-------------+---------+------+----------+---+---------+------------------------------+---------------

In [5]:
df_receipt.show(3)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
only showing top 3 rows



In [21]:
df_customer.join(
    df_receipt.select("customer_id", "amount"),
    "customer_id",
    "left"
).filter(
    (F.col("gender_cd") == 1 ) &
    (~F.col("customer_id").startswith("Z"))
).groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).fillna(0).show(10)

+--------------+------------+
|   customer_id|total_amount|
+--------------+------------+
|CS028413000240|           0|
|CS004313000147|           0|
|CS049513000008|        4450|
|CS024415000195|        4638|
|CS032613000113|           0|
|CS027413000061|           0|
|CS017514000012|        4699|
|CS002415000856|           0|
|CS010413000069|         278|
|CS004413000815|           0|
+--------------+------------+
only showing top 10 rows



PysparkでNULLを任意の値で埋めるにはdataframe.fillna()を用いる。  
カラム単位でNULLを埋めたい場合はwhenとColumn.isNullを併用するしかないかも（下記）。  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameNaFunctions.fill.html?highlight=fill%20null

In [15]:
df_customer.join(
    df_receipt.select("customer_id", "amount"),
    "customer_id",
    "left"
).filter(
    (F.col("gender_cd") == 1 ) &
    (~F.col("customer_id").startswith("Z"))
).groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).withColumn(
    "total_amount",
    F.when(F.col("total_amount").isNull(), 0
    ).otherwise(F.col("total_amount"))
).show(10)

+--------------+------------+
|   customer_id|total_amount|
+--------------+------------+
|CS028413000240|           0|
|CS004313000147|           0|
|CS049513000008|        4450|
|CS024415000195|        4638|
|CS032613000113|           0|
|CS027413000061|           0|
|CS017514000012|        4699|
|CS002415000856|           0|
|CS010413000069|         278|
|CS004413000815|           0|
+--------------+------------+
only showing top 10 rows



---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、  
> 売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [71]:
df_1 = spark.createDataFrame(
    df_receipt.filter(
        ~F.col("customer_id").startswith("Z")
    ).groupBy("customer_id").agg(
        F.count_distinct(F.col("sales_ymd")).alias("counts_sales_ymd")
    ).sort("counts_sales_ymd", ascending=False).head(20)
)

In [61]:
df_2 = \
(spark.createDataFrame(
    df_receipt.filter(
        ~F.col("customer_id").startswith("Z")
    ).groupBy("customer_id").agg(
        F.sum(F.col("amount")).alias("total_amount")
    ).sort("total_amount", ascending=False).head(20)
 )
)

### 重要
.show()は標準出力に吐かれてしまうので、DataFrameとして一部抜き出す用途には使えない。  
.head()は先頭から指定した行数抜き出すことは可能だが、結果がリストで返ってくるため、  
この後にjoinするにはspark.createDataFrameでDataFrameに変換しておく必要がある。

In [72]:
df_1.join(
    df_2,
    "customer_id",
    "full_outer"
).sort("counts_sales_ymd", ascending=False).show(20)

+--------------+----------------+------------+
|   customer_id|counts_sales_ymd|total_amount|
+--------------+----------------+------------+
|CS040214000008|              23|        NULL|
|CS010214000010|              22|       18585|
|CS015415000185|              22|       20153|
|CS010214000002|              21|        NULL|
|CS028415000007|              21|       19127|
|CS016415000141|              20|       18372|
|CS017415000097|              20|       23086|
|CS014214000023|              19|        NULL|
|CS021514000045|              19|        NULL|
|CS021515000172|              19|        NULL|
|CS022515000226|              19|        NULL|
|CS031414000051|              19|       19202|
|CS039414000052|              19|        NULL|
|CS007515000107|              18|        NULL|
|CS014415000077|              18|        NULL|
|CS021515000056|              18|        NULL|
|CS021515000211|              18|        NULL|
|CS022515000028|              18|        NULL|
|CS0302140000

---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [64]:
df_store.show(3)

+--------+----------+-------------+----------+------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                       address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S12014|  千草台店|           12|    千葉県|千葉県千葉市稲毛区千草台一丁目|   チバケンチバシイナゲクチグサダイイ...|043-123-4003|  140.118|35.63559|    1698.0|
|  S13002|  国分寺店|           13|    東京都|      東京都国分寺市本多二丁目|トウキョウトコクブンジシホンダニチョウメ|042-123-4008| 139.4802|35.70566|    1735.0|
|  S14010|    菊名店|           14|  神奈川県|神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
+--------+----------+-------------+----------+------------------------------+----------------------------------------+------------+---------+----

In [66]:
df_product.show(3)

+----------+-----------------+------------------+-----------------+----------+---------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|
+----------+-----------------+------------------+-----------------+----------+---------+
|P040101001|               04|              0401|           040101|       198|      149|
|P040101002|               04|              0401|           040101|       218|      164|
|P040101003|               04|              0401|           040101|       230|      173|
+----------+-----------------+------------------+-----------------+----------+---------+
only showing top 3 rows



store,productは共通のキーがないので、１店舗ごとに全ての製品が紐づくようにキーを作成すればよい。(直積)  
※直積：集合AとBに対して、Aの要素aとBの要素bを組にした (a, b) の形の全組み合わせの集合を作ること。A×Bで表わす。

In [78]:
df_store.withColumn(
    "key",
    F.lit(0)
).join(
    df_product.withColumn("key", F.lit(0)),
    "key",
    "left"
).count()

531590

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [93]:
# windowの定義
w = Window.orderBy("sales_ymd")

df_receipt.groupBy("sales_ymd").agg(
    F.sum(F.col("amount")).alias("total_amount")
).withColumn(
    "diff_amount",
    # F.lagはoffset(ここでは１)の分だけ前の行の値を返すwindow関数
    # この結果を元々のカラムから引くことでpolarsのshiftに相当する操作ができる。
    F.col("total_amount") - F.lag("total_amount", 1).over(w)
).show(10)

+---------+------------+-----------+
|sales_ymd|total_amount|diff_amount|
+---------+------------+-----------+
| 20170101|       33723|       NULL|
| 20170102|       24165|      -9558|
| 20170103|       27503|       3338|
| 20170104|       36165|       8662|
| 20170105|       37830|       1665|
| 20170106|       32387|      -5443|
| 20170107|       23415|      -8972|
| 20170108|       24737|       1322|
| 20170109|       26718|       1981|
| 20170110|       20143|      -6575|
+---------+------------+-----------+
only showing top 10 rows



polarsのようなshiftはないので、Window関数で求める。

---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、  
> 各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [105]:
w = Window.orderBy("sales_ymd")

df_receipt.groupBy("sales_ymd").agg(
    F.sum(F.col("amount")).alias("total_amount")
).withColumns({
    "total_amount_yesterday": F.lag("total_amount").over(w),
    "total_amount_2_days_ago": F.lag("total_amount", 2).over(w),
    "total_amount_3_days_ago": F.lag("total_amount", 3).over(w),
}).show(10)

+---------+------------+----------------------+-----------------------+-----------------------+
|sales_ymd|total_amount|total_amount_yesterday|total_amount_2_days_ago|total_amount_3_days_ago|
+---------+------------+----------------------+-----------------------+-----------------------+
| 20170101|       33723|                  NULL|                   NULL|                   NULL|
| 20170102|       24165|                 33723|                   NULL|                   NULL|
| 20170103|       27503|                 24165|                  33723|                   NULL|
| 20170104|       36165|                 27503|                  24165|                  33723|
| 20170105|       37830|                 36165|                  27503|                  24165|
| 20170106|       32387|                 37830|                  36165|                  27503|
| 20170107|       23415|                 32387|                  37830|                  36165|
| 20170108|       24737|                

Window関数を用いることで、pandasに比べて非常にきれいに書ける。

---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。  
> 性別コードは0が男性、1が女性、9が不明を表すものとする。  
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [106]:
df_receipt.show(3)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
only showing top 3 rows



In [107]:
df_customer.show(3)

+--------------+-------------+---------+------+----------+---+---------+------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                       address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+------------------------------+--------------------+----------------+------------+
|CS021313000114|  大野 あや子|        1|  女性|1981-04-29| 37| 259-1113|神奈川県伊勢原市粟窪**********|              S14021|        20150905|0-00000000-0|
|CS037613000071|    六角 雅彦|        9|  不明|1952-04-01| 66| 136-0076|    東京都江東区南砂**********|              S13037|        20150414|0-00000000-0|
|CS031415000172|宇多田 貴美子|        1|  女性|1976-10-04| 42| 151-0053|  東京都渋谷区代々木**********|              S13031|        20150529|D-20100325-C|
+--------------+-------------+---------+------+----------+---+---------+------------------------------+---------------

In [126]:
df_receipt.join(
    df_customer.withColumn(
        "age_group",
        F.floor(F.col("age") / 10) * 10 # 小数点以下切り捨て
    ),
    "customer_id",
    "inner"
).select(
    "gender_cd",
    "age_group",
    "amount"
).groupBy("age_group").pivot("gender_cd").sum(
    "amount"
).sort("age_group").show()



+---------+------+-------+------+
|age_group|     0|      1|     9|
+---------+------+-------+------+
|       10|  1591| 149836|  4317|
|       20| 72940|1363724| 44328|
|       30|177322| 693047| 50441|
|       40| 19355|9320791|483512|
|       50| 54320|6685192|342923|
|       60|272469| 987741| 71418|
|       70| 13435|  29764|  2427|
|       80| 46360| 262923|  5111|
|       90|  NULL|   6260|  NULL|
+---------+------+-------+------+



pysparkでクロス集計を行うにはgroupby（縦軸）とpivot（横軸）を組み合わせる。

---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。  
> このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [12]:
df_receipt.join(
    df_customer.withColumn(
        "age_group",
        F.floor(F.col("age") / 10) * 10 # 小数点以下切り捨て
    ),
    "customer_id",
    "inner"
).select(
    "gender_cd",
    "age_group",
    "amount"
).groupBy("age_group", "gender_cd").agg(
    F.sum("amount")
).withColumn(
    "gender_cd",
    F.when(F.col("gender_cd") == 0, "00"
    ).when(F.col("gender_cd") == 1, "01"
    ).when(F.col("gender_cd") == 9, "99")
).sort("age_group", "gender_cd").show()



+---------+---------+-----------+
|age_group|gender_cd|sum(amount)|
+---------+---------+-----------+
|       10|       00|       1591|
|       10|       01|     149836|
|       10|       99|       4317|
|       20|       00|      72940|
|       20|       01|    1363724|
|       20|       99|      44328|
|       30|       00|     177322|
|       30|       01|     693047|
|       30|       99|      50441|
|       40|       00|      19355|
|       40|       01|    9320791|
|       40|       99|     483512|
|       50|       00|      54320|
|       50|       01|    6685192|
|       50|       99|     342923|
|       60|       00|     272469|
|       60|       01|     987741|
|       60|       99|      71418|
|       70|       00|      13435|
|       70|       01|      29764|
+---------+---------+-----------+
only showing top 20 rows



---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [14]:
df_customer.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- gender_cd: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_day: date (nullable = true)
 |-- age: long (nullable = true)
 |-- postal_cd: string (nullable = true)
 |-- address: string (nullable = true)
 |-- application_store_cd: string (nullable = true)
 |-- application_date: string (nullable = true)
 |-- status_cd: string (nullable = true)



### 方法1：Date->stringにキャストしてから整形

In [20]:
df_customer.withColumn(
    "birth_day",
    F.regexp_replace(F.col("birth_day").cast("string"), "-", "")
).select(
    "customer_id",
    "birth_day"
).show(10)

+--------------+---------+
|   customer_id|birth_day|
+--------------+---------+
|CS021313000114| 19810429|
|CS037613000071| 19520401|
|CS031415000172| 19761004|
|CS028811000001| 19330327|
|CS001215000145| 19950329|
|CS020401000016| 19740915|
|CS015414000103| 19770809|
|CS029403000008| 19730817|
|CS015804000004| 19310502|
|CS033513000180| 19620711|
+--------------+---------+
only showing top 10 rows



### 方法2：date_format関数を用いる

In [25]:
df_customer.withColumn(
    "birth_day",
    F.date_format("birth_day", "yyyyMMdd")
).select(
    "customer_id",
    "birth_day"
).show(10)

+--------------+---------+
|   customer_id|birth_day|
+--------------+---------+
|CS021313000114| 19810429|
|CS037613000071| 19520401|
|CS031415000172| 19761004|
|CS028811000001| 19330327|
|CS001215000145| 19950329|
|CS020401000016| 19740915|
|CS015414000103| 19770809|
|CS029403000008| 19730817|
|CS015804000004| 19310502|
|CS033513000180| 19620711|
+--------------+---------+
only showing top 10 rows



---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。  
> これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [31]:
df_customer.withColumn(
    "application_date",
    F.to_date("application_date", "yyyyMMdd")
).select(
    "customer_id",
    "application_date"
).show(10)

+--------------+----------------+
|   customer_id|application_date|
+--------------+----------------+
|CS021313000114|      2015-09-05|
|CS037613000071|      2015-04-14|
|CS031415000172|      2015-05-29|
|CS028811000001|      2016-01-15|
|CS001215000145|      2017-06-05|
|CS020401000016|      2015-02-25|
|CS015414000103|      2015-07-22|
|CS029403000008|      2015-05-15|
|CS015804000004|      2015-06-07|
|CS033513000180|      2015-07-28|
+--------------+----------------+
only showing top 10 rows



formatにはpolarsでなじみのある"%Y%m%d"は使えないので注意。  
[参考：formatの指定方法](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html)

---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。  
> これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [37]:
df_receipt.select(
    "receipt_no",
    "receipt_sub_no",
    F.to_date("sales_ymd", "yyyyMMdd").alias("sales_ymd")
).show(10)

+----------+--------------+----------+
|receipt_no|receipt_sub_no| sales_ymd|
+----------+--------------+----------+
|       112|             1|2018-11-03|
|      1132|             2|2018-11-18|
|      1102|             1|2017-07-12|
|      1132|             1|2019-02-05|
|      1102|             2|2018-08-21|
|      1112|             1|2019-06-05|
|      1102|             2|2018-12-05|
|      1102|             1|2019-09-22|
|      1112|             2|2017-05-04|
|      1102|             1|2019-10-10|
+----------+--------------+----------+
only showing top 10 rows



---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。  
> これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [40]:
df_receipt.show(3)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
only showing top 3 rows



In [41]:
df_receipt.select(
    "receipt_no",
    "receipt_sub_no",
    F.to_date(F.from_unixtime("sales_epoch"), "yyyy-MM-dd HH:mm:ss").alias("sales_time")
).show(10)

+----------+--------------+----------+
|receipt_no|receipt_sub_no|sales_time|
+----------+--------------+----------+
|       112|             1|2018-11-03|
|      1132|             2|2018-11-18|
|      1102|             1|2017-07-12|
|      1132|             1|2019-02-05|
|      1102|             2|2018-08-21|
|      1112|             1|2019-06-05|
|      1102|             2|2018-12-05|
|      1102|             1|2019-09-22|
|      1112|             2|2017-05-04|
|      1102|             1|2019-10-10|
+----------+--------------+----------+
only showing top 10 rows



---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、  
> 「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [45]:
df_receipt.select(
    "receipt_no",
    "receipt_sub_no",
    F.year(
        F.to_date(F.from_unixtime("sales_epoch"), "yyyy-MM-dd HH:mm:ss")
    ).alias("sales_year")
).show(10)

+----------+--------------+----------+
|receipt_no|receipt_sub_no|sales_year|
+----------+--------------+----------+
|       112|             1|      2018|
|      1132|             2|      2018|
|      1102|             1|      2017|
|      1132|             1|      2019|
|      1102|             2|      2018|
|      1112|             1|      2019|
|      1102|             2|      2018|
|      1102|             1|      2019|
|      1112|             2|      2017|
|      1102|             1|      2019|
+----------+--------------+----------+
only showing top 10 rows



---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、  
> レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [ ]:
df_receipt.select(
    "receipt_no",
    "receipt_sub_no",
    F.month(
        F.to_date(F.from_unixtime("sales_epoch"), "yyyy-MM-dd HH:mm:ss")
    ).alias("sales_month")
).withColumn(
    "sales_month",
    # left padding
    F.lpad(F.col("sales_month").cast("string"), 2, "0")
).show(10)

+----------+--------------+-----------+
|receipt_no|receipt_sub_no|sales_month|
+----------+--------------+-----------+
|       112|             1|         11|
|      1132|             2|         11|
|      1102|             1|         07|
|      1132|             1|         02|
|      1102|             2|         08|
|      1112|             1|         06|
|      1102|             2|         12|
|      1102|             1|         09|
|      1112|             2|         05|
|      1102|             1|         10|
+----------+--------------+-----------+
only showing top 10 rows



### 別解：from_unixtimeの結果を整形する  

In [73]:
df_receipt.select(
    "receipt_no",
    "receipt_sub_no",
    # yyyy-MM-dd HH:mm:ssを-および半角スペースで分割して、月の部分だけ取り出す
    (F.split(F.from_unixtime("sales_epoch").alias("sales_month"), "-|\s")[1]).alias("sales_month")
).show(10)

+----------+--------------+-----------+
|receipt_no|receipt_sub_no|sales_month|
+----------+--------------+-----------+
|       112|             1|         11|
|      1132|             2|         11|
|      1102|             1|         07|
|      1132|             1|         02|
|      1102|             2|         08|
|      1112|             1|         06|
|      1102|             2|         12|
|      1102|             1|         09|
|      1112|             2|         05|
|      1102|             1|         10|
+----------+--------------+-----------+
only showing top 10 rows



---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、  
> レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [74]:
df_receipt.select(
    "receipt_no",
    "receipt_sub_no",
    # yyyy-MM-dd HH:mm:ssを-および半角スペースで分割して、月の部分だけ取り出す
    (F.split(F.from_unixtime("sales_epoch").alias("sales_month"), "-|\s")[2]).alias("sales_month")
).show(10)

+----------+--------------+-----------+
|receipt_no|receipt_sub_no|sales_month|
+----------+--------------+-----------+
|       112|             1|         03|
|      1132|             2|         18|
|      1102|             1|         12|
|      1132|             1|         05|
|      1102|             2|         21|
|      1112|             1|         05|
|      1102|             2|         05|
|      1102|             1|         22|
|      1112|             2|         04|
|      1102|             1|         10|
+----------+--------------+-----------+
only showing top 10 rows



---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、  
> 売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。  
> ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [82]:
df_receipt.filter(
    ~F.col("customer_id").startswith("Z")
).groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).withColumn(
    "is_total_amount_over_2000",
    F.when(F.col("total_amount") > 2000, 1
    ).otherwise(0)
).sort("customer_id").show(10)

+--------------+------------+-------------------------+
|   customer_id|total_amount|is_total_amount_over_2000|
+--------------+------------+-------------------------+
|CS001113000004|        1298|                        0|
|CS001114000005|         626|                        0|
|CS001115000010|        3044|                        1|
|CS001205000004|        1988|                        0|
|CS001205000006|        3337|                        1|
|CS001211000025|         456|                        0|
|CS001212000027|         448|                        0|
|CS001212000031|         296|                        0|
|CS001212000046|         228|                        0|
|CS001212000070|         456|                        0|
+--------------+------------+-------------------------+
only showing top 10 rows



---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。  
> さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [83]:
df_customer.show(2)

+--------------+-------------+---------+------+----------+---+---------+------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                       address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+------------------------------+--------------------+----------------+------------+
|CS021313000114|  大野 あや子|        1|  女性|1981-04-29| 37| 259-1113|神奈川県伊勢原市粟窪**********|              S14021|        20150905|0-00000000-0|
|CS037613000071|    六角 雅彦|        9|  不明|1952-04-01| 66| 136-0076|    東京都江東区南砂**********|              S13037|        20150414|0-00000000-0|
+--------------+-------------+---------+------+----------+---+---------+------------------------------+--------------------+----------------+------------+
only showing top 2 rows



In [97]:
df_customer.withColumn(
    "is_target_postal_cd",
    F.when(F.regexp("postal_cd", F.lit(r"(^1[0-9][0-9])|(^20[0-9])")), 1).otherwise(0)
).filter(
    (F.col("postal_cd").startswith("2") ) & (F.col("is_target_postal_cd") == 1)
).show(5)

+--------------+-----------------+---------+------+----------+---+---------+----------------------------+--------------------+----------------+------------+-------------------+
|   customer_id|    customer_name|gender_cd|gender| birth_day|age|postal_cd|                     address|application_store_cd|application_date|   status_cd|is_target_postal_cd|
+--------------+-----------------+---------+------+----------+---+---------+----------------------------+--------------------+----------------+------------+-------------------+
|CS003613000188|      西本 めぐみ|        1|  女性|1950-05-18| 68| 201-0012|東京都狛江市中和泉**********|              S13003|        20160820|0-00000000-0|                  1|
|CS003615000432|宇野 サンタマリア|        1|  女性|1951-02-09| 68| 201-0012|東京都狛江市中和泉**********|              S13003|        20170716|0-00000000-0|                  1|
|CS041515000001|        栗田 千夏|        1|  女性|1967-01-02| 52| 206-0001|  東京都多摩市和田**********|              S13041|        20160422|E-20100803-F|        

確かにマッチしている

In [102]:
df_receipt.show(3)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
only showing top 3 rows



In [105]:
df_customer.withColumn(
    "is_target_postal_cd",
    F.when(F.regexp("postal_cd", F.lit(r"(^1[0-9][0-9])|(^20[0-9])")), 1).otherwise(0)
).join(
    df_receipt,
    "customer_id",
    "inner"
).groupby("is_target_postal_cd").agg(
    F.count_distinct("customer_id").alias("customer_count")
).show(10)

+-------------------+--------------+
|is_target_postal_cd|customer_count|
+-------------------+--------------+
|                  1|          4400|
|                  0|          3906|
+-------------------+--------------+



join後の時点で、全期間において売り上げ実績のある（df_receiptにある）顧客と紐づいているので、  
あとは顧客数を重複なくカウントすればよい。ここでは購入回数は気にしない。

---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、  
> 顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [107]:
prefecture_cd = [("埼玉県", 11), ("千葉県", 12), ("東京都", 13), ("神奈川県", 14)]

In [111]:
# シンプルに書くためにexprをチェインして書いておく
expr = F
for prefecture, cd in prefecture_cd:
    expr = expr.when(F.col("address").contains(prefecture), cd)

In [113]:
df_customer.withColumn(
    "prefec_cd",
    expr
).select(
    "customer_id",
    "address",
    "prefec_cd"
).show(10)

+--------------+--------------------------------+---------+
|   customer_id|                         address|prefec_cd|
+--------------+--------------------------------+---------+
|CS021313000114|  神奈川県伊勢原市粟窪**********|       14|
|CS037613000071|      東京都江東区南砂**********|       13|
|CS031415000172|    東京都渋谷区代々木**********|       13|
|CS028811000001|神奈川県横浜市泉区和泉町*****...|       14|
|CS001215000145|    東京都大田区仲六郷**********|       13|
|CS020401000016|      東京都板橋区若木**********|       13|
|CS015414000103|      東京都江東区北砂**********|       13|
|CS029403000008|      千葉県浦安市海楽**********|       12|
|CS015804000004|      東京都江東区北砂**********|       13|
|CS033513000180|神奈川県横浜市旭区善部町*****...|       14|
+--------------+--------------------------------+---------+
only showing top 10 rows



---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。  
> その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [133]:
# 顧客ごとに合計を計算
df_recept_total = df_receipt.groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
)

In [129]:
# 四分位数を計算。実はリストもアンパックできる
q1, q2, q3 = df_recept_total.approxQuantile("total_amount", [0.25, 0.5, 0.75], 0.0)

In [138]:
q1, q2, q3

(548.0, 1478.0, 3652.0)

[approxQuantile](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.approxQuantile.html?highlight=approxquantile)  
このメソッドはリストで四分位数を返してくれる。ちょっと特殊な方法で四分位数を概算している模様。

In [139]:
df_recept_total.withColumn(
    "amount_category",
    F.when(F.col("total_amount") < q1, 1
    ).when(F.col("total_amount") < q2, 2
    ).when(F.col("total_amount") < q3, 3
    ).otherwise(4)
).sort("customer_id").show(10)

+--------------+------------+---------------+
|   customer_id|total_amount|amount_category|
+--------------+------------+---------------+
|CS001113000004|        1298|              2|
|CS001114000005|         626|              2|
|CS001115000010|        3044|              3|
|CS001205000004|        1988|              3|
|CS001205000006|        3337|              3|
|CS001211000025|         456|              1|
|CS001212000027|         448|              1|
|CS001212000031|         296|              1|
|CS001212000046|         228|              1|
|CS001212000070|         456|              1|
+--------------+------------+---------------+
only showing top 10 rows



---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。  
> ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [143]:
df_customer.select(
    "customer_id",
    "birth_day",
    "age"
).withColumn(
    "age_group",
    F.when(F.col("age") >= 60, 60
    ).otherwise(
        F.floor(F.col("age") / 10) * 10
    )
).show(10)

+--------------+----------+---+---------+
|   customer_id| birth_day|age|age_group|
+--------------+----------+---+---------+
|CS021313000114|1981-04-29| 37|       30|
|CS037613000071|1952-04-01| 66|       60|
|CS031415000172|1976-10-04| 42|       40|
|CS028811000001|1933-03-27| 86|       60|
|CS001215000145|1995-03-29| 24|       20|
|CS020401000016|1974-09-15| 44|       40|
|CS015414000103|1977-08-09| 41|       40|
|CS029403000008|1973-08-17| 45|       40|
|CS015804000004|1931-05-02| 87|       60|
|CS033513000180|1962-07-11| 56|       50|
+--------------+----------+---+---------+
only showing top 10 rows



---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。  
> 組み合わせを表すカテゴリの値は任意とする。

In [144]:
df_customer.select(
    "customer_id",
    "birth_day",
    "age",
    "gender_cd"
).withColumn(
    "age_group",
    F.when(F.col("age") >= 60, 60
    ).otherwise(
        F.floor(F.col("age") / 10) * 10
    )
).withColumn(
    "gender_and_age_group",
    F.concat_ws("_", "gender_cd", "age_group")
).show(10)

+--------------+----------+---+---------+---------+--------------------+
|   customer_id| birth_day|age|gender_cd|age_group|gender_and_age_group|
+--------------+----------+---+---------+---------+--------------------+
|CS021313000114|1981-04-29| 37|        1|       30|                1_30|
|CS037613000071|1952-04-01| 66|        9|       60|                9_60|
|CS031415000172|1976-10-04| 42|        1|       40|                1_40|
|CS028811000001|1933-03-27| 86|        1|       60|                1_60|
|CS001215000145|1995-03-29| 24|        1|       20|                1_20|
|CS020401000016|1974-09-15| 44|        0|       40|                0_40|
|CS015414000103|1977-08-09| 41|        1|       40|                1_40|
|CS029403000008|1973-08-17| 45|        0|       40|                0_40|
|CS015804000004|1931-05-02| 87|        0|       60|                0_60|
|CS033513000180|1962-07-11| 56|        1|       50|                1_50|
+--------------+----------+---+---------+---------+

---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

PySparkにおけるOne-Hot Encodingは、まずStringIndexerを使ってカテゴリ列を数値インデックスに変換し、  
その後OneHotEncoderを使ってOne Hot Encodingを行う必要がある。  
※One-Hot EncodingのtransformメソッドではStringをそのままOne-Hotの形に変換できないため、一度数値インデックスに変換する必要がある。

In [221]:
string_indexer = StringIndexer(inputCol="gender_cd", outputCol="gender_cd_idx")
model = string_indexer.fit(df_customer)
indexed = model.transform(df_customer)

# One hot encoding
ohe = OneHotEncoder(inputCol="gender_cd_idx", outputCol="gender_cd_ohe")
ohe = ohe.fit(indexed)
ohe = ohe.transform(indexed)

In [225]:
ohe.select(
    "customer_id",
    "gender_cd",
    "gender_cd_idx",
    "gender_cd_ohe"
).show()

+--------------+---------+-------------+-------------+
|   customer_id|gender_cd|gender_cd_idx|gender_cd_ohe|
+--------------+---------+-------------+-------------+
|CS021313000114|        1|          0.0|(2,[0],[1.0])|
|CS037613000071|        9|          2.0|    (2,[],[])|
|CS031415000172|        1|          0.0|(2,[0],[1.0])|
|CS028811000001|        1|          0.0|(2,[0],[1.0])|
|CS001215000145|        1|          0.0|(2,[0],[1.0])|
|CS020401000016|        0|          1.0|(2,[1],[1.0])|
|CS015414000103|        1|          0.0|(2,[0],[1.0])|
|CS029403000008|        0|          1.0|(2,[1],[1.0])|
|CS015804000004|        0|          1.0|(2,[1],[1.0])|
|CS033513000180|        1|          0.0|(2,[0],[1.0])|
|CS007403000016|        0|          1.0|(2,[1],[1.0])|
|CS035614000014|        1|          0.0|(2,[0],[1.0])|
|CS011215000048|        1|          0.0|(2,[0],[1.0])|
|CS009413000079|        1|          0.0|(2,[0],[1.0])|
|CS040412000191|        1|          0.0|(2,[0],[1.0])|
|CS0294150

PySparkにおけるOne-Hot Encodingの結果は、pandasやpolarsにおけるOne-Hot Encodingとは見た目が異なる出力になる。  
これはSparseVectorと呼ばれ、SparseVectorはPySparkにおいて、ベクトルが疎である場合にメモリ効率を高めるために使用される。  
この形式は次のように解釈すればよい。  
1. 最初の数値はベクトルの長さを表す
2. 真ん中の中括弧内の数値は非ゼロ要素のインデックスを表す
3. 最後の中括弧内の数値は対応する非ゼロ要素の値を表す

例えば、
* (2,[0],[1.0]) は、長さ2のベクトルで、インデックス0に1.0があることを示す。（つまり、ベクトルは [1.0, 0.0]）。
* (2,[],[]) は、長さ2のベクトルで、すべての要素が0であることを示す。（つまり、ベクトルは [0.0, 0.0]）。
    * gender_cdは３つのカテゴリカル変数であり、うち２つ(0, 1)がゼロであるとわかれば、残り(9)は表示しなくてもわかるので省略されている。
* (2,[1],[1.0]) は、長さ2のベクトルで、インデックス1に1.0があることを示している。（つまり、ベクトルは [0.0, 1.0]）。

[sparse vectorの説明](https://mogile.web.fc2.com/spark/spark160/mllib-data-types.html)
>->例えば、ベクトル (1.0, 0.0, 3.0) は [1.0, 0.0, 3.0]としてdense形式、あるいは (3, [0, 2], [1.0, 3.0]) 3はベクトルのサイズとしてsparse形式で表すことができます。

### - SparseVectorをカラムに分割する方法

In [189]:
ohe_mod = ohe.withColumn(
    "gender_cd_ohe", 
    vector_to_array("gender_cd_ohe") # SparseVectorを配列に変換する
)

In [201]:
# カテゴリの数をカウント。gender_cd=9はなくてもよいのでカウント外。
num_categories = len(ohe_mod.select("gender_cd_ohe").head()[0])
num_categories

2

In [202]:
for i in range(num_categories):
    ohe_mod = ohe_mod.withColumn(
        f"category_{i}",
        ohe_mod["gender_cd_ohe"].getItem(i) # getItemでリスト内の対応するインデックスの値を取得できる
    )

[getItem](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.getItem.html?highlight=getitem)

In [215]:
ohe_mod.select(
    "customer_id",
    "gender_cd_ohe",
    "category_0",
    "category_1",
).show()

+--------------+-------------+----------+----------+
|   customer_id|gender_cd_ohe|category_0|category_1|
+--------------+-------------+----------+----------+
|CS021313000114|   [1.0, 0.0]|       1.0|       0.0|
|CS037613000071|   [0.0, 0.0]|       0.0|       0.0|
|CS031415000172|   [1.0, 0.0]|       1.0|       0.0|
|CS028811000001|   [1.0, 0.0]|       1.0|       0.0|
|CS001215000145|   [1.0, 0.0]|       1.0|       0.0|
|CS020401000016|   [0.0, 1.0]|       0.0|       1.0|
|CS015414000103|   [1.0, 0.0]|       1.0|       0.0|
|CS029403000008|   [0.0, 1.0]|       0.0|       1.0|
|CS015804000004|   [0.0, 1.0]|       0.0|       1.0|
|CS033513000180|   [1.0, 0.0]|       1.0|       0.0|
|CS007403000016|   [0.0, 1.0]|       0.0|       1.0|
|CS035614000014|   [1.0, 0.0]|       1.0|       0.0|
|CS011215000048|   [1.0, 0.0]|       1.0|       0.0|
|CS009413000079|   [1.0, 0.0]|       1.0|       0.0|
|CS040412000191|   [1.0, 0.0]|       1.0|       0.0|
|CS029415000023|   [1.0, 0.0]|       1.0|     

### - whenでOne-Hot Encodingする
カテゴリ数が大して多くなければこっちの方が楽かも。

In [219]:
df_customer.select(
    "customer_id",
    "gender_cd"
).withColumns({
    "category_0": 
    F.when(F.col("gender_cd") == 0, 1).otherwise(0),
    "category_1": 
    F.when(F.col("gender_cd") == 1, 1).otherwise(0),
}).show()

+--------------+---------+----------+----------+
|   customer_id|gender_cd|category_0|category_1|
+--------------+---------+----------+----------+
|CS021313000114|        1|         0|         1|
|CS037613000071|        9|         0|         0|
|CS031415000172|        1|         0|         1|
|CS028811000001|        1|         0|         1|
|CS001215000145|        1|         0|         1|
|CS020401000016|        0|         1|         0|
|CS015414000103|        1|         0|         1|
|CS029403000008|        0|         1|         0|
|CS015804000004|        0|         1|         0|
|CS033513000180|        1|         0|         1|
|CS007403000016|        0|         1|         0|
|CS035614000014|        1|         0|         1|
|CS011215000048|        1|         0|         1|
|CS009413000079|        1|         0|         1|
|CS040412000191|        1|         0|         1|
|CS029415000023|        1|         0|         1|
|CS009315000023|        1|         0|         1|
|CS040702000012|    

### - gender_cdのユニーク値を確認

In [284]:
[v["gender_cd"] for v in df_customer.select("gender_cd").distinct().collect()]

['0', '9', '1']

In [261]:
df_customer.select("gender_cd").dropDuplicates().rdd.flatMap(lambda x: x).collect()

['0', '9', '1']

---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、  
> 売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。  
> 標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [305]:
# まず売り上げ金額合計のDataframeを作成する
df_amount = df_receipt.filter(
    ~F.col("customer_id").startswith("Z")
).groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
)

In [311]:
# 平均値、標準偏差(母標準偏差)を計算
amount_mean = df_amount.select(
    F.mean("total_amount")
).collect()[0][0]

amount_std = df_amount.select(
    F.stddev_pop("total_amount")
).collect()[0][0]

In [313]:
df_amount.withColumn(
    "standardized_total_amount",
    (F.col("total_amount") - amount_mean) / amount_std
).orderBy("customer_id").show(10)

+--------------+------------+-------------------------+
|   customer_id|total_amount|standardized_total_amount|
+--------------+------------+-------------------------+
|CS001113000004|        1298|     -0.45937788025813364|
|CS001114000005|         626|      -0.7063903657165813|
|CS001115000010|        3044|      0.18241348820979741|
|CS001205000004|        1988|     -0.20574898893919183|
|CS001205000006|        3337|       0.2901138724945075|
|CS001211000025|         456|      -0.7688786432879149|
|CS001212000027|         448|      -0.7718192681148012|
|CS001212000031|         296|      -0.8276911398256405|
|CS001212000046|         228|      -0.8526864508541739|
|CS001212000070|         456|      -0.7688786432879149|
+--------------+------------+-------------------------+
only showing top 10 rows



---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、  
> 売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_amount = df_receipt.filter(
    ~F.col("customer_id").startswith("Z")
).groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
)

In [314]:
# 最大値、最小値を計算
amount_max = df_amount.select(
    F.max("total_amount")
).collect()[0][0]

amount_min = df_amount.select(
    F.min("total_amount")
).collect()[0][0]

In [318]:
df_amount.withColumn(
    "normalized_total_amount",
    F.round((F.col("total_amount") - amount_min) / (amount_max - amount_min), 4)
).sort("customer_id").show(10)

+--------------+------------+-----------------------+
|   customer_id|total_amount|normalized_total_amount|
+--------------+------------+-----------------------+
|CS001113000004|        1298|                 0.0534|
|CS001114000005|         626|                 0.0242|
|CS001115000010|        3044|                 0.1292|
|CS001205000004|        1988|                 0.0833|
|CS001205000006|        3337|                 0.1419|
|CS001211000025|         456|                 0.0168|
|CS001212000027|         448|                 0.0164|
|CS001212000031|         296|                 0.0098|
|CS001212000046|         228|                 0.0069|
|CS001212000070|         456|                 0.0168|
+--------------+------------+-----------------------+
only showing top 10 rows



---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、  
> 売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [321]:
df_amount = df_receipt.filter(
    ~F.col("customer_id").startswith("Z")
).groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).withColumn(
    "log10_total_amount",
    F.log10(F.col("total_amount"))
).sort("customer_id").show(10)

+--------------+------------+------------------+
|   customer_id|total_amount|log10_total_amount|
+--------------+------------+------------------+
|CS001113000004|        1298|3.1132746924643504|
|CS001114000005|         626|2.7965743332104296|
|CS001115000010|        3044|3.4834446480985353|
|CS001205000004|        1988|3.2984163800612945|
|CS001205000006|        3337|3.5233562066547925|
|CS001211000025|         456|2.6589648426644352|
|CS001212000027|         448| 2.651278013998144|
|CS001212000031|         296|2.4712917110589387|
|CS001212000046|         228| 2.357934847000454|
|CS001212000070|         456|2.6589648426644352|
+--------------+------------+------------------+
only showing top 10 rows



---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。  
> ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [322]:
df_amount = df_receipt.filter(
    ~F.col("customer_id").startswith("Z")
).groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).withColumn(
    "log_e_total_amount",
    F.log(F.col("total_amount"))
).sort("customer_id").show(10)

+--------------+------------+------------------+
|   customer_id|total_amount|log_e_total_amount|
+--------------+------------+------------------+
|CS001113000004|        1298| 7.168579897264035|
|CS001114000005|         626| 6.439350371100098|
|CS001115000010|        3044| 8.020927718981577|
|CS001205000004|        1988|  7.59488438721652|
|CS001205000006|        3337| 8.112827478751374|
|CS001211000025|         456|6.1224928095143865|
|CS001212000027|         448| 6.104793232414985|
|CS001212000031|         296|  5.69035945432406|
|CS001212000046|         228| 5.429345628954441|
|CS001212000070|         456|6.1224928095143865|
+--------------+------------+------------------+
only showing top 10 rows



---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [3]:
df_product.withColumn(
    "unit_profit",
    F.col("unit_price") - F.col("unit_cost")
).show(10)

+----------+-----------------+------------------+-----------------+----------+---------+-----------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|unit_profit|
+----------+-----------------+------------------+-----------------+----------+---------+-----------+
|P040101001|               04|              0401|           040101|       198|      149|         49|
|P040101002|               04|              0401|           040101|       218|      164|         54|
|P040101003|               04|              0401|           040101|       230|      173|         57|
|P040101004|               04|              0401|           040101|       248|      186|         62|
|P040101005|               04|              0401|           040101|       268|      201|         67|
|P040101006|               04|              0401|           040101|       298|      224|         74|
|P040101007|               04|              0401|           040101|       338|      254|   

---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。  
> ただし、単価と原価には欠損が生じていることに注意せよ。

In [17]:
# df_product.filter(
#     (F.col("unit_price").isNotNull()) &
#     (F.col("unit_cost").isNotNull())
# )
df_product.select(
    F.mean((F.col("unit_price") - F.col("unit_cost")) / F.col("unit_price"))
).collect()[0][0]

0.24911389885176904

計算対象のカラムに欠損値が含まれていても無視して計算できる模様。

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。  
> そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [23]:
df_product.withColumn(
    # F.mean((F.col("unit_price") - F.col("unit_cost")) / F.col("unit_price"))
    "new_unit_price",
    F.floor((F.col("unit_cost") / 0.7))
).withColumn(
    "new_profit_rate",
    F.round((F.col("new_unit_price") - F.col("unit_cost")) / F.col("new_unit_price"), 6)
).show(10)

+----------+-----------------+------------------+-----------------+----------+---------+--------------+---------------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|new_unit_price|new_profit_rate|
+----------+-----------------+------------------+-----------------+----------+---------+--------------+---------------+
|P040101001|               04|              0401|           040101|       198|      149|           212|        0.29717|
|P040101002|               04|              0401|           040101|       218|      164|           234|       0.299145|
|P040101003|               04|              0401|           040101|       230|      173|           247|       0.299595|
|P040101004|               04|              0401|           040101|       248|      186|           265|       0.298113|
|P040101005|               04|              0401|           040101|       268|      201|           287|       0.299652|
|P040101006|               04|          

---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。  
> 今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。  
> ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [28]:
df_product.withColumn(
    "new_unit_price",
    F.round(((F.col("unit_cost") / 0.7)), 0)
).withColumn(
    "new_profit_rate",
    F.round((F.col("new_unit_price") - F.col("unit_cost")) / F.col("new_unit_price"), 6)
).show(10)

+----------+-----------------+------------------+-----------------+----------+---------+--------------+---------------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|new_unit_price|new_profit_rate|
+----------+-----------------+------------------+-----------------+----------+---------+--------------+---------------+
|P040101001|               04|              0401|           040101|       198|      149|         213.0|       0.300469|
|P040101002|               04|              0401|           040101|       218|      164|         234.0|       0.299145|
|P040101003|               04|              0401|           040101|       230|      173|         247.0|       0.299595|
|P040101004|               04|              0401|           040101|       248|      186|         266.0|       0.300752|
|P040101005|               04|              0401|           040101|       268|      201|         287.0|       0.299652|
|P040101006|               04|          

---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。  
> そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [29]:
df_product.withColumn(
    "new_unit_price",
    F.ceil(((F.col("unit_cost") / 0.7)))
).withColumn(
    "new_profit_rate",
    F.round((F.col("new_unit_price") - F.col("unit_cost")) / F.col("new_unit_price"), 6)
).show(10)

+----------+-----------------+------------------+-----------------+----------+---------+--------------+---------------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|new_unit_price|new_profit_rate|
+----------+-----------------+------------------+-----------------+----------+---------+--------------+---------------+
|P040101001|               04|              0401|           040101|       198|      149|           213|       0.300469|
|P040101002|               04|              0401|           040101|       218|      164|           235|       0.302128|
|P040101003|               04|              0401|           040101|       230|      173|           248|       0.302419|
|P040101004|               04|              0401|           040101|       248|      186|           266|       0.300752|
|P040101005|               04|              0401|           040101|       268|      201|           288|       0.302083|
|P040101006|               04|          

---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。  
> ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [33]:
df_product.withColumn(
    "tax_price",
    F.floor(F.col("unit_price") * 1.1)
).show(10)

+----------+-----------------+------------------+-----------------+----------+---------+---------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|tax_price|
+----------+-----------------+------------------+-----------------+----------+---------+---------+
|P040101001|               04|              0401|           040101|       198|      149|      217|
|P040101002|               04|              0401|           040101|       218|      164|      239|
|P040101003|               04|              0401|           040101|       230|      173|      253|
|P040101004|               04|              0401|           040101|       248|      186|      272|
|P040101005|               04|              0401|           040101|       268|      201|      294|
|P040101006|               04|              0401|           040101|       298|      224|      327|
|P040101007|               04|              0401|           040101|       338|      254|      371|
|P04010100

---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、  
> カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。  
> 抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [34]:
df_product.show(1)

+----------+-----------------+------------------+-----------------+----------+---------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|
+----------+-----------------+------------------+-----------------+----------+---------+
|P040101001|               04|              0401|           040101|       198|      149|
+----------+-----------------+------------------+-----------------+----------+---------+
only showing top 1 row



In [3]:
# 結合
receipt_and_product = df_receipt.join(
    df_product,
    "product_cd",
    "left"
)

# 顧客ごとの全商品の売上金額合計計算
amount_all = receipt_and_product.groupBy("customer_id").agg(
    F.sum("amount").alias("amount_all")
)

# 07の売上金額合計計算
amount_07 = receipt_and_product.filter(
    F.col("category_major_cd") == "07" # この時点で07の購入実績のある顧客に絞られる
).groupBy("customer_id").agg(
    F.sum("amount").alias("amount_07")
)

# 結合して比率を計算。
amount_all.join(
    amount_07,
    "customer_id",
    "inner"
).withColumn(
    "ratio",
    F.round(F.col("amount_07") / F.col("amount_all"), 3)
).sort("customer_id").show(10)



+--------------+----------+---------+-----+
|   customer_id|amount_all|amount_07|ratio|
+--------------+----------+---------+-----+
|CS001113000004|      1298|     1298|  1.0|
|CS001114000005|       626|      486|0.776|
|CS001115000010|      3044|     2694|0.885|
|CS001205000004|      1988|      346|0.174|
|CS001205000006|      3337|     2004|0.601|
|CS001212000027|       448|      200|0.446|
|CS001212000031|       296|      296|  1.0|
|CS001212000046|       228|      108|0.474|
|CS001212000070|       456|      308|0.675|
|CS001213000018|       243|      145|0.597|
+--------------+----------+---------+-----+
only showing top 10 rows



---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、  
> 顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [32]:
df_receipt.join(
    df_customer,
    "customer_id",
    "left"
).select(
    "customer_id",
    "sales_ymd",
    F.to_date("sales_ymd", "yyyyMMdd").alias("sales_ymd_2"),
    "application_date",
    F.to_date("application_date", "yyyyMMdd").alias("application_date_2")
).withColumn(
    "days_from_application",
    F.datediff("sales_ymd_2", "application_date_2")
).filter(F.col("customer_id") == "CS006214000001").show(10)

+--------------+---------+-----------+----------------+------------------+---------------------+
|   customer_id|sales_ymd|sales_ymd_2|application_date|application_date_2|days_from_application|
+--------------+---------+-----------+----------------+------------------+---------------------+
|CS006214000001| 20181103| 2018-11-03|        20150201|        2015-02-01|                 1371|
|CS006214000001| 20170509| 2017-05-09|        20150201|        2015-02-01|                  828|
|CS006214000001| 20170608| 2017-06-08|        20150201|        2015-02-01|                  858|
|CS006214000001| 20170608| 2017-06-08|        20150201|        2015-02-01|                  858|
|CS006214000001| 20181028| 2018-10-28|        20150201|        2015-02-01|                 1365|
|CS006214000001| 20181028| 2018-10-28|        20150201|        2015-02-01|                 1365|
|CS006214000001| 20170509| 2017-05-09|        20150201|        2015-02-01|                  828|
|CS006214000001| 20190908| 201

2つのDate型の値の日数差は"datediff"で求められる。  
解答と合わせるために顧客IDでフィルタリングしている。  
yyyyMMddを誤ってyyyymmddにすると、間違った日時になるので注意。  


---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、  
> 顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [36]:
# １か月は３０日と考える
days_of_month = 30

df_receipt.join(
    df_customer,
    "customer_id",
    "left"
).select(
    "customer_id",
    "sales_ymd",
    F.to_date("sales_ymd", "yyyyMMdd").alias("sales_ymd_2"),
    "application_date",
    F.to_date("application_date", "yyyyMMdd").alias("application_date_2")
).withColumn(
    "days_from_application",
    F.datediff("sales_ymd_2", "application_date_2")
).withColumn(
    "months_from_application",
    F.floor(F.col("days_from_application") / 30)
).filter(F.col("customer_id") == "CS006214000001").show(10)

+--------------+---------+-----------+----------------+------------------+---------------------+-----------------------+
|   customer_id|sales_ymd|sales_ymd_2|application_date|application_date_2|days_from_application|months_from_application|
+--------------+---------+-----------+----------------+------------------+---------------------+-----------------------+
|CS006214000001| 20181103| 2018-11-03|        20150201|        2015-02-01|                 1371|                     45|
|CS006214000001| 20170509| 2017-05-09|        20150201|        2015-02-01|                  828|                     27|
|CS006214000001| 20170608| 2017-06-08|        20150201|        2015-02-01|                  858|                     28|
|CS006214000001| 20170608| 2017-06-08|        20150201|        2015-02-01|                  858|                     28|
|CS006214000001| 20181028| 2018-10-28|        20150201|        2015-02-01|                 1365|                     45|
|CS006214000001| 20181028| 2018-

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、  
> 顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [38]:
# １か月は３０日と考える
days_of_month = 30

df_receipt.join(
    df_customer,
    "customer_id",
    "left"
).select(
    "customer_id",
    "sales_ymd",
    F.to_date("sales_ymd", "yyyyMMdd").alias("sales_ymd_2"),
    "application_date",
    F.to_date("application_date", "yyyyMMdd").alias("application_date_2")
).withColumn(
    "days_from_application",
    F.datediff("sales_ymd_2", "application_date_2")
).withColumn(
    "years_from_application",
    F.floor(F.col("days_from_application") / 365)
).filter(F.col("customer_id") == "CS006214000001").show(10)

+--------------+---------+-----------+----------------+------------------+---------------------+----------------------+
|   customer_id|sales_ymd|sales_ymd_2|application_date|application_date_2|days_from_application|years_from_application|
+--------------+---------+-----------+----------------+------------------+---------------------+----------------------+
|CS006214000001| 20181103| 2018-11-03|        20150201|        2015-02-01|                 1371|                     3|
|CS006214000001| 20170509| 2017-05-09|        20150201|        2015-02-01|                  828|                     2|
|CS006214000001| 20170608| 2017-06-08|        20150201|        2015-02-01|                  858|                     2|
|CS006214000001| 20170608| 2017-06-08|        20150201|        2015-02-01|                  858|                     2|
|CS006214000001| 20181028| 2018-10-28|        20150201|        2015-02-01|                 1365|                     3|
|CS006214000001| 20181028| 2018-10-28|  

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、  
> 顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。  
> なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [45]:
df_receipt.printSchema()

root
 |-- sales_ymd: long (nullable = true)
 |-- sales_epoch: long (nullable = true)
 |-- store_cd: string (nullable = true)
 |-- receipt_no: long (nullable = true)
 |-- receipt_sub_no: long (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_cd: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- amount: long (nullable = true)



In [51]:
df_receipt.join(
    df_customer,
    "customer_id",
    "left"
).select(
    "customer_id",
    "sales_epoch",
    F.to_unix_timestamp(
        F.to_date("application_date", "yyyyMMdd")
    ).alias("application_date_epoch")
).withColumn(
    "elapsed_epoch",
    F.col("sales_epoch") - F.col("application_date_epoch")
).filter(F.col("customer_id") == "CS006214000001").show(10)

+--------------+-----------+----------------------+-------------+
|   customer_id|sales_epoch|application_date_epoch|elapsed_epoch|
+--------------+-----------+----------------------+-------------+
|CS006214000001| 1541203200|            1422748800|    118454400|
|CS006214000001| 1494288000|            1422748800|     71539200|
|CS006214000001| 1496880000|            1422748800|     74131200|
|CS006214000001| 1496880000|            1422748800|     74131200|
|CS006214000001| 1540684800|            1422748800|    117936000|
|CS006214000001| 1540684800|            1422748800|    117936000|
|CS006214000001| 1494288000|            1422748800|     71539200|
|CS006214000001| 1567900800|            1422748800|    145152000|
|CS006214000001| 1517356800|            1422748800|     94608000|
|CS006214000001| 1499212800|            1422748800|     76464000|
+--------------+-----------+----------------------+-------------+
only showing top 10 rows



---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、  
> 売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [59]:
df_receipt.select(
    F.to_date("sales_ymd", "yyyyMMdd").alias("sales_ymd")
).withColumn(
    "days_from_last_Monday",
    F.when(
        F.dayofweek("sales_ymd") == 1, 6 # dayofweekでは日曜が1なので、6に変換
    ).otherwise(F.dayofweek("sales_ymd") - 2)  # dayofweekの2は月曜日
).withColumn(
    "last_Monday",
    F.date_add("sales_ymd", -F.col("days_from_last_Monday"))
).show(10)

+----------+---------------------+-----------+
| sales_ymd|days_from_last_Monday|last_Monday|
+----------+---------------------+-----------+
|2018-11-03|                    5| 2018-10-29|
|2018-11-18|                    6| 2018-11-12|
|2017-07-12|                    2| 2017-07-10|
|2019-02-05|                    1| 2019-02-04|
|2018-08-21|                    1| 2018-08-20|
|2019-06-05|                    2| 2019-06-03|
|2018-12-05|                    2| 2018-12-03|
|2019-09-22|                    6| 2019-09-16|
|2017-05-04|                    3| 2017-05-01|
|2019-10-10|                    3| 2019-10-07|
+----------+---------------------+-----------+
only showing top 10 rows



Date型に対する日付の足し匹はdate_add関数を用いる。

---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [61]:
df_customer.count()

21971

In [70]:
df_customer.sample(0.01).count()

226

厳密に1%の行数が返ってくるわけではない。  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.sample.html?highlight=samp#pyspark.sql.DataFrame.sample

---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、  
> 性別コードごとに件数を集計せよ。

In [3]:
df_customer.groupBy("gender_cd").agg(
    F.count("customer_id").alias("counts")
).withColumn(
    "ratio",
    F.round(F.col("counts") / df_customer.count(), 3)
).show()

+---------+------+-----+
|gender_cd|counts|ratio|
+---------+------+-----+
|        0|  2981|0.136|
|        9|  1072|0.049|
|        1| 17918|0.816|
+---------+------+-----+



In [4]:
df_customer.sampleBy(
    "gender_cd", fractions={"0": 0.1, "9": 0.1, "1": 0.1}
).groupBy("gender_cd").agg(
    F.count("customer_id").alias("counts")
).show()

+---------+------+
|gender_cd|counts|
+---------+------+
|        0|   279|
|        9|   106|
|        1|  1789|
+---------+------+



---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。  
> なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [14]:
total_amounts = df_receipt.groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).withColumn(
    "total_amount_log",
    F.log("total_amount")
)

total_amounts.show(5)

+--------------+------------+-----------------+
|   customer_id|total_amount| total_amount_log|
+--------------+------------+-----------------+
|CS024415000195|        4638|8.442038517815478|
|CS011515000044|        2533|7.837159650001675|
|CS017415000245|        1289|7.161622002939187|
|CS011512000113|         686|6.530877627725885|
|CS025414000093|        3929| 8.27614021955846|
+--------------+------------+-----------------+
only showing top 5 rows



In [31]:
mean_and_std = total_amounts.select(
    F.mean("total_amount_log").alias("mean"),
    F.std("total_amount_log").alias("std")
).collect()

mean, std = mean_and_std[0][0], mean_and_std[0][1]

In [32]:
mean_and_std

[Row(mean=7.270295238032214, std=1.1379898752498037)]

In [36]:
total_amounts.filter(
    F.col("total_amount_log") > (mean + 3 * std)
).show(10)

+--------------+------------+------------------+
|   customer_id|total_amount|  total_amount_log|
+--------------+------------+------------------+
|ZZ000000000000|    12395003|16.332803965484477|
+--------------+------------+------------------+



### ※別解
一律同じ値のカラムを作り、Windowを使って強引に平均値と標準偏差のカラムを作ることで、  
余計な変数を作らずに算出できる。

In [51]:
df_receipt.groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).withColumns({
    "total_amount_log":
    F.log("total_amount"),
    "key": # partition定義用のカラム
    F.lit(1)
}).withColumns({
    "mean":
    F.mean("total_amount_log").over(Window.partitionBy("key")), # 全行を対象に集計する
    "std":
    F.std("total_amount_log").over(Window.partitionBy("key")),
}).show(5)

+--------------+------------+-----------------+---+-----------------+------------------+
|   customer_id|total_amount| total_amount_log|key|             mean|               std|
+--------------+------------+-----------------+---+-----------------+------------------+
|CS024415000195|        4638|8.442038517815478|  1|7.270295238032214|1.1379898752498037|
|CS011515000044|        2533|7.837159650001675|  1|7.270295238032214|1.1379898752498037|
|CS017415000245|        1289|7.161622002939187|  1|7.270295238032214|1.1379898752498037|
|CS011512000113|         686|6.530877627725885|  1|7.270295238032214|1.1379898752498037|
|CS025414000093|        3929| 8.27614021955846|  1|7.270295238032214|1.1379898752498037|
+--------------+------------+-----------------+---+-----------------+------------------+
only showing top 5 rows



In [48]:
df_receipt.groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).withColumns({
    "total_amount_log":
    F.log("total_amount"),
    "key":
    F.lit(1)
}).withColumns({
    "mean":
    F.mean("total_amount_log").over(Window.partitionBy("key")),
    "std":
    F.std("total_amount_log").over(Window.partitionBy("key")),
}).filter(
    F.col("total_amount_log") > (F.col("mean") + F.col("std") * 3)
).show(10)

+--------------+------------+------------------+---+-----------------+------------------+
|   customer_id|total_amount|  total_amount_log|key|             mean|               std|
+--------------+------------+------------------+---+-----------------+------------------+
|ZZ000000000000|    12395003|16.332803965484477|  1|7.270295238032214|1.1379898752498037|
+--------------+------------+------------------+---+-----------------+------------------+



---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。  
> ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。  
> なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。  

In [59]:
df_receipt.filter(
    ~F.col("customer_id").startswith("Z")
).groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
).withColumn(
    "key",
    F.lit(1)
).withColumns({
    "1st_quartile":
    F.percentile("total_amount", 0.25).over(Window.partitionBy("key")),
    "3rd_quartile":
    F.percentile("total_amount", 0.75).over(Window.partitionBy("key")),
}).withColumn(
    "IQR",
    F.col("3rd_quartile") - F.col("1st_quartile")
).filter(
    ( F.col("total_amount") < (F.col("1st_quartile") - 1.5 * F.col("IQR")) ) |
    ( F.col("total_amount") > (F.col("1st_quartile") + 1.5 * F.col("IQR")) )
).sort("customer_id").show(10)

+--------------+------------+---+------------+------------+------+
|   customer_id|total_amount|key|1st_quartile|3rd_quartile|   IQR|
+--------------+------------+---+------------+------------+------+
|CS001214000052|        5639|  1|      548.25|     3649.75|3101.5|
|CS001412000160|        5646|  1|      548.25|     3649.75|3101.5|
|CS001414000048|        8584|  1|      548.25|     3649.75|3101.5|
|CS001414000113|        6343|  1|      548.25|     3649.75|3101.5|
|CS001414000172|        5394|  1|      548.25|     3649.75|3101.5|
|CS001415000102|        6497|  1|      548.25|     3649.75|3101.5|
|CS001415000480|        6635|  1|      548.25|     3649.75|3101.5|
|CS001515000320|        5815|  1|      548.25|     3649.75|3101.5|
|CS001605000009|       18925|  1|      548.25|     3649.75|3101.5|
|CS002214000043|        8073|  1|      548.25|     3649.75|3101.5|
+--------------+------------+---+------------+------------+------+
only showing top 10 rows



---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [70]:
for col in df_product.columns:
    print(
        f"{col}: {df_product.filter(F.col(col).isNull()).count()}"
    )

product_cd: 0
category_major_cd: 0
category_medium_cd: 0
category_small_cd: 0
unit_price: 7
unit_cost: 7


---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。  
> なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [77]:
print(f"削除前：{df_product.count()}")
print(f"削除後：{df_product.dropna().count()}")

削除前：10030
削除後：10023


In [74]:
for col in df_product.columns:
    print(
        f"{col}: {df_product.dropna().filter(F.col(col).isNull()).count()}"
    )

product_cd: 0
category_major_cd: 0
category_medium_cd: 0
category_small_cd: 0
unit_price: 0
unit_cost: 0


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。  
> なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [3]:
price_mean = df_product.select(F.round(F.mean("unit_price"), 0)).collect()[0][0]
cost_mean = df_product.select(F.round(F.mean("unit_cost"), 0)).collect()[0][0]
price_mean, cost_mean

(403.0, 302.0)

In [4]:
df_product.filter(
    (F.col("unit_price").isNull()) |  (F.col("unit_cost").isNull())
).show(10)

+----------+-----------------+------------------+-----------------+----------+---------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|
+----------+-----------------+------------------+-----------------+----------+---------+
|P040802007|               04|              0408|           040802|      NULL|     NULL|
|P050103021|               05|              0501|           050103|      NULL|     NULL|
|P050405009|               05|              0504|           050405|      NULL|     NULL|
|P060802026|               06|              0608|           060802|      NULL|     NULL|
|P070202092|               07|              0702|           070202|      NULL|     NULL|
|P080504027|               08|              0805|           080504|      NULL|     NULL|
|P090204185|               09|              0902|           090204|      NULL|     NULL|
+----------+-----------------+------------------+-----------------+----------+---------+



In [5]:
# 欠損値が埋まったかの確認用に控えておく
target_rows = df_product.filter(
    (F.col("unit_price").isNull()) |  (F.col("unit_cost").isNull())
).select("product_cd")

target_rows = [v["product_cd"] for v in target_rows.select("product_cd").distinct().collect()]

In [6]:
fillnulled = df_product.withColumns({
    "unit_price":
    F.when(F.col("unit_price").isNull(), price_mean).otherwise(F.col("unit_price")),
    "unit_cost":
    F.when(F.col("unit_cost").isNull(), cost_mean).otherwise(F.col("unit_cost")),
})

In [7]:
fillnulled.count()

10030

欠損はない

In [8]:
fillnulled.filter(
    F.col("product_cd").isin(target_rows)
).show(10)

+----------+-----------------+------------------+-----------------+----------+---------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|
+----------+-----------------+------------------+-----------------+----------+---------+
|P040802007|               04|              0408|           040802|     403.0|    302.0|
|P050103021|               05|              0501|           050103|     403.0|    302.0|
|P050405009|               05|              0504|           050405|     403.0|    302.0|
|P060802026|               06|              0608|           060802|     403.0|    302.0|
|P070202092|               07|              0702|           070202|     403.0|    302.0|
|P080504027|               08|              0805|           080504|     403.0|    302.0|
|P090204185|               09|              0902|           090204|     403.0|    302.0|
+----------+-----------------+------------------+-----------------+----------+---------+



確かに欠損値が平均値で埋められている。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。  
> なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [24]:
price_median = df_product.select(F.round(F.median("unit_price"))).collect()[0][0]
cost_median = df_product.select(F.round(F.median("unit_cost"))).collect()[0][0]
price_median, cost_median

(252.0, 189.0)

In [29]:
fillnulled = df_product.withColumns({
    "unit_price":
    F.when(F.col("unit_price").isNull(), price_median).otherwise(F.col("unit_price")),
    "unit_cost":
    F.when(F.col("unit_cost").isNull(), cost_median).otherwise(F.col("unit_cost"))
})

fillnulled.filter(
    F.col("product_cd").isin(target_rows)
).show()

+----------+-----------------+------------------+-----------------+----------+---------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|
+----------+-----------------+------------------+-----------------+----------+---------+
|P040802007|               04|              0408|           040802|     252.0|    189.0|
|P050103021|               05|              0501|           050103|     252.0|    189.0|
|P050405009|               05|              0504|           050405|     252.0|    189.0|
|P060802026|               06|              0608|           060802|     252.0|    189.0|
|P070202092|               07|              0702|           070202|     252.0|    189.0|
|P080504027|               08|              0805|           080504|     252.0|    189.0|
|P090204185|               09|              0902|           090204|     252.0|    189.0|
+----------+-----------------+------------------+-----------------+----------+---------+



In [31]:
for col in ["unit_price", "unit_cost"]:
    print(f"{col}: {fillnulled.filter(F.col(col).isNull()).count()}")

unit_price: 0
unit_cost: 0


欠損値がないことが確認できた。

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。  
> なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [37]:
new_price_and_cost = df_product.groupBy("category_small_cd").agg(
    F.round(F.median("unit_price")).alias("unit_price"),
    F.round(F.median("unit_cost")).alias("unit_cost"),
)
new_price_and_cost.show(5)

+-----------------+----------+---------+
|category_small_cd|unit_price|unit_cost|
+-----------------+----------+---------+
|           040101|     283.0|    213.0|
|           040102|     378.0|    284.0|
|           040201|     223.0|    168.0|
|           040202|     178.0|    134.0|
|           040203|     308.0|    231.0|
+-----------------+----------+---------+
only showing top 5 rows



In [42]:
df_product.filter(
    # unit_price, unit_costが欠損している行でフィルタリング
    F.col("product_cd").isin(target_rows)
).drop(
    # 結合後不要なので落とす
    "unit_price", "unit_cost"
).join(
    new_price_and_cost,
    "category_small_cd",
    "left",
).show()

+-----------------+----------+-----------------+------------------+----------+---------+
|category_small_cd|product_cd|category_major_cd|category_medium_cd|unit_price|unit_cost|
+-----------------+----------+-----------------+------------------+----------+---------+
|           040802|P040802007|               04|              0408|     313.0|    235.0|
|           050103|P050103021|               05|              0501|     133.0|    100.0|
|           050405|P050405009|               05|              0504|     178.0|    134.0|
|           060802|P060802026|               06|              0608|     270.0|    200.0|
|           070202|P070202092|               07|              0702|     238.0|    179.0|
|           080504|P080504027|               08|              0805|     258.0|    196.0|
|           090204|P090204185|               09|              0902|     694.0|    521.0|
+-----------------+----------+-----------------+------------------+----------+---------+



---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。  
> ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [53]:
df_amount_2019 = df_customer.join(
    df_receipt.filter(
        F.col("sales_ymd") >= 20190000
    ),
    "customer_id",
    "left"
).groupBy("customer_id").agg(
    F.sum("amount").alias("amount_2019")
).withColumn(
    "amount_2019",
    F.when(F.col("amount_2019").isNull(), 0).otherwise(F.col("amount_2019"))
)

df_amount_all = df_customer.join(
    df_receipt,
    "customer_id",
    "left"
).groupBy("customer_id").agg(
    F.sum("amount").alias("amount_all")
).withColumn(
    "amount_all",
    F.when(F.col("amount_all").isNull(), 0).otherwise(F.col("amount_all"))
)


In [73]:
df_result = df_amount_all.join(
    df_amount_2019,
    "customer_id",
    "inner"
).withColumn(
    "amount_ratio",
    F.round(F.col("amount_2019") / F.col("amount_all"), 2)
).filter(
    F.col("amount_ratio") > 0
).sort("customer_id")

df_result.show(10)

+--------------+----------+-----------+------------+
|   customer_id|amount_all|amount_2019|amount_ratio|
+--------------+----------+-----------+------------+
|CS001113000004|      1298|       1298|         1.0|
|CS001114000005|       626|        188|         0.3|
|CS001115000010|      3044|        578|        0.19|
|CS001205000004|      1988|        702|        0.35|
|CS001205000006|      3337|        486|        0.15|
|CS001211000025|       456|        456|         1.0|
|CS001212000070|       456|        456|         1.0|
|CS001214000009|      4685|        664|        0.14|
|CS001214000017|      4132|       2962|        0.72|
|CS001214000048|      2374|       1889|         0.8|
+--------------+----------+-----------+------------+
only showing top 10 rows



In [78]:
for col in df_result.columns:
    print(f"{col}: {df_result.filter(F.col(col).isNull()).count()}")

customer_id: 0
amount_all: 0
amount_2019: 0
amount_ratio: 0


---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。  
> ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。  
> また、作成結果を確認するために結果を10件表示せよ。

In [93]:
df_geocode.groupBy("postal_cd").agg(
    F.count("postal_cd").alias("count")
).orderBy("count", ascending=False).show(5)

+---------+-----+
|postal_cd|count|
+---------+-----+
| 452-0961|   66|
| 480-1103|   65|
| 771-2102|   49|
| 921-8046|   48|
| 441-0302|   46|
+---------+-----+
only showing top 5 rows



一つの郵便番号に複数のレコードが紐づくことはある模様

In [96]:
new_longitude_latitude = df_geocode.groupBy("postal_cd").agg(
    F.mean("longitude").alias("longitude"),
    F.mean("latitude").alias("latitude")
)
new_longitude_latitude.show(5)

+---------+----------+---------+
|postal_cd| longitude| latitude|
+---------+----------+---------+
| 064-0825|141.314886|43.062472|
| 005-0855| 141.34241| 42.93771|
| 005-0016| 141.35046| 42.98468|
| 063-0029| 141.21742| 43.05001|
| 004-0004| 141.47742| 43.04422|
+---------+----------+---------+
only showing top 5 rows



In [98]:
df_customer.join(
    df_geocode,
    "postal_cd",
    "left"
).show(10)

+---------+--------------+-------------+---------+------+----------+---+--------------------------------+--------------------+----------------+------------+----------+----------+------+------+-------+------------------------+---------+--------+
|postal_cd|   customer_id|customer_name|gender_cd|gender| birth_day|age|                         address|application_store_cd|application_date|   status_cd|prefecture|      city|  town|street|address|            full_address|longitude|latitude|
+---------+--------------+-------------+---------+------+----------+---+--------------------------------+--------------------+----------------+------------+----------+----------+------+------+-------+------------------------+---------+--------+
| 259-1113|CS021313000114|  大野 あや子|        1|  女性|1981-04-29| 37|  神奈川県伊勢原市粟窪**********|              S14021|        20150905|0-00000000-0|  神奈川県|  伊勢原市|  粟窪|  NULL|   NULL|    神奈川県伊勢原市粟窪|139.31779|35.41358|
| 136-0076|CS037613000071|    六角 雅彦|        9|  不明|1952-0